In [ ]:
!nvidia-smi

Wed Jul  7 03:09:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install missingpy

In [ ]:
pip install neural-tangents

In [ ]:
# from MICE import *
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
from torch.autograd import Variable
from torch.nn import functional as F
from matplotlib.ticker import MultipleLocator
from matplotlib.pyplot import figure
import statsmodels.api as sm
import time

import jax.numpy
import neural_tangents as nt
from neural_tangents import stax
from jax import random
# from jax.experimental import optimizers
# from jax.api import jit, grad, vmap
import functools

import math
import copy
import statistics
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import *
from sklearn.svm import *

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df = pd.read_csv('/content/drive/My Drive/Data/adni_org.csv')
df = df.drop(['PTGENDER','DX_bl','PTRACCAT'], axis=1)
df = df.drop(df.columns[0], axis=1)
print(df)

        8355  126282   92736  284099  ...   26791   8683  595098  MNI_Hippocampus_R
0    2.30525  5.1490  2.4235   2.268  ...  11.887  9.430  11.826           0.467067
1    2.19000  4.8195  2.4490   2.130  ...  12.063  9.734  11.952           0.421698
2    2.23900  5.4440  2.7675   1.957  ...  11.506  9.578  11.949           0.383664
3    2.25450  5.3955  2.6865   2.347  ...  11.240  9.504  11.901           0.452964
4    2.18650  5.3405  2.5875   2.154  ...  11.524  9.865  11.959           0.427152
..       ...     ...     ...     ...  ...     ...    ...     ...                ...
644  2.19550  5.5090  2.6485   2.261  ...  11.640  9.508  11.905           0.354836
645  2.31375  5.1385  2.8110   2.201  ...  11.585  9.666  11.882           0.440644
646  2.32375  5.1670  2.5785   2.205  ...  11.395  9.401  11.969           0.506813
647  2.33300  5.5065  2.4100   2.161  ...  11.769  9.368  11.942           0.403784
648  2.37150  5.5250  2.6460   2.130  ...  11.691  9.342  11.923           0

In [ ]:
scaler = StandardScaler(with_mean=True, with_std=False).fit(df.iloc[:,0:-1])
df.iloc[:,0:-1] = scaler.transform(df.iloc[:,0:-1])
print(df)

         8355   126282     92736  ...      8683  595098  MNI_Hippocampus_R
0    0.016239 -0.18876 -0.128827  ... -0.160445 -0.0531           0.467067
1   -0.099011 -0.51826 -0.103327  ...  0.143555  0.0729           0.421698
2   -0.050011  0.10624  0.215173  ... -0.012445  0.0699           0.383664
3   -0.034511  0.05774  0.134173  ... -0.086445  0.0219           0.452964
4   -0.102511  0.00274  0.035173  ...  0.274555  0.0799           0.427152
..        ...      ...       ...  ...       ...     ...                ...
644 -0.093511  0.17124  0.096173  ... -0.082445  0.0259           0.354836
645  0.024739 -0.19926  0.258673  ...  0.075555  0.0029           0.440644
646  0.034739 -0.17076  0.026173  ... -0.189445  0.0899           0.506813
647  0.043989  0.16874 -0.142327  ... -0.222445  0.0629           0.403784
648  0.082489  0.18724  0.093673  ... -0.248445  0.0439           0.477733

[649 rows x 19823 columns]


In [ ]:
dfy = df['MNI_Hippocampus_R']
cor = np.zeros(19822)
for i in range(19822):
  dfx_col = df.iloc[:, i] 
  cor[i] = dfx_col.corr(dfy)

In [ ]:
data = df.to_numpy()
p_f=p=1500
X = data[:,cor.argsort()[-p:][::-1]]
y = data[:,-1].reshape((-1,1))
print(X.shape, y.shape)
adni_data = np.concatenate((X, y), axis=1)
print(adni_data.shape)

(649, 1500) (649, 1)
(649, 1501)


In [ ]:
n=adni_data.shape[0]

use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if use_cuda else 'cpu')
print(device)

np.random.seed(1)
true_beta=np.zeros(p)
q=[0,1,2]
# q=[0,1,2,3,4]

cuda:0


In [ ]:
def data_G_linear(n,p,data,seed=1,a0=0,a1=0.1,a2=0.1,a3=0,a4=0.1,a5=0):
  np.random.seed(seed) 
  sample = data.copy()
  missing_col1 = np.zeros(p+1)
  missing_col2 = np.zeros(p+1)
  if p==1000:
    missing_col1[0:200]=1
    missing_col2[200:400]=1
  elif p==1500:
    missing_col1[0:200]=1
    missing_col2[200:400]=1
  elif p==10000:
    missing_col1[0:1000]=1
    missing_col2[1000:2000]=1
    # missing_col1[0:100]=1
    # missing_col1[600:1000]=1
    # missing_col2[1500:2000]=1
  elif p==19822:
    # missing_col1[0:2000]=1
    # missing_col2[2000:4000]=1
    missing_col1[0:1000]=1
    missing_col2[1000:2000]=1
  y=sample[:,-1]
  ###  MNAR  ###
  # logit1=a0+a1*np.mean(sample[:,missing_col2==1][:,0:5],axis=1)+a2*y
  # logit2=a3+a4*np.mean(sample[:,missing_col1==1][:,0:5],axis=1)+a5*y
  ###  MAR  ###
  logit1=a0+a1*np.mean(sample[:,np.logical_or(missing_col1,missing_col2)==0][:,0:100],axis=1)+a2*y
  logit2=a3+a4*np.mean(sample[:,np.logical_or(missing_col1,missing_col2)==0][:,200:300],axis=1)+a5*y
  p_miss1 = 1/(1+np.exp(-logit1))  
  p_miss2 = 1/(1+np.exp(-logit2))  
  missing_row1=np.zeros((n,))
  missing_row2=np.zeros((n,))
  for i in range(n):
    missing_row1[i]= np.random.choice(2, 1, p=[1-p_miss1[i],p_miss1[i]])
    missing_row2[i]= np.random.choice(2, 1, p=[1-p_miss2[i],p_miss2[i]])

  return sample,missing_row1==1,missing_row2==1,missing_col1==1,missing_col2==1,y.reshape((-1,1))

In [ ]:
def data_to_stats(imputed_data_list,truth1,truth2,rows1,rows2,cols1,cols2):
  # for real data analysis, there is no true_bata
  rounds=len(imputed_data_list)
  MSE=0
  MEANs=np.zeros((rounds,4))
  within_VAR=np.zeros((4,4))
  # MEANs=np.zeros((rounds,6))
  # within_VAR=np.zeros((6,6))

  for i in range(rounds):
    imputed_data_MICE=imputed_data_list[i]
    X=imputed_data_MICE[:,:-1]
    ols = sm.OLS(imputed_data_MICE[:,-1], sm.add_constant(X[:,[0,1,2]]))
    # ols = sm.OLS(imputed_data_MICE[:,-1], sm.add_constant(X[:,[0,1,2,3,4]]))
    result=ols.fit()
    within_VAR+=result.cov_params()/rounds
    MEANs[i,:]=result.params
    MSE+=np.sum((imputed_data_MICE[rows1,:][:,cols1]-truth1)**2)/(rounds*(np.sum(rows1)*np.sum(cols1)+np.sum(rows2)*np.sum(cols2)))
    MSE+=np.sum((imputed_data_MICE[rows2,:][:,cols2]-truth2)**2)/(rounds*(np.sum(rows1)*np.sum(cols1)+np.sum(rows2)*np.sum(cols2)))
  
  if rounds==1:
    VAR=within_VAR
    MEAN=MEANs[0,:]
  else:
    # Rubin's rule for total variance
    MEAN=np.mean(MEANs,axis=0)
    Betw_VAR=np.zeros((4,4))
    # Betw_VAR=np.zeros((6,6))
    # between imputation variance
    for i in range(rounds):
        temp=MEANs[i,:]-MEAN
        Betw_VAR+=np.outer(temp,temp)/(rounds-1)
    VAR=within_VAR+(1+1/rounds)*Betw_VAR
  return MSE,MEAN,VAR,np.sqrt(VAR[1][1])

# ***benchmark***

In [ ]:
full_SD=[]
partial_SD=[]
naive_imp_SD=[]

full_SE=0
partial_SE=0
naive_imp_SE=0

naive_imp_MSE=0

full_beta_estimate=0
partial_beta_estimate=0
naive_imp_beta_estimate=0

num_of_missing_row = 0
num_of_missing_row1 = 0
num_of_missing_row2 = 0

a0=-1
a1=-3
a2=3
a3=-1
a4=-3
a5=2
Monte_Carlo =100

for iteration in range(1,1+Monte_Carlo):
  sample,missing_row1,missing_row2,missing_col1,missing_col2,y=data_G_linear(n=n,p=p,data=adni_data,seed=iteration,a0=a0,a1=a1,a2=a2,a3=a3,a4=a4,a5=a5)
  # print(sum(missing_row))
  truth1=sample[missing_row1==True,:][:,missing_col1==True].copy()
  truth2=sample[missing_row2==True,:][:,missing_col2==True].copy()

  complete_data=sample.copy()
  complete_case=sample[np.logical_or(missing_row1,missing_row2)==0,:].copy()
  # print(complete_case.shape)
  
  num_mis = np.sum(np.logical_or(missing_row1, missing_row2))
  num_mis1 = np.sum(missing_row1)
  num_mis2 = np.sum(missing_row2)
  num_of_missing_row1 = (num_of_missing_row1*(iteration-1)+num_mis1)/iteration
  num_of_missing_row2 = (num_of_missing_row2*(iteration-1)+num_mis2)/iteration
  num_of_missing_row = (num_of_missing_row*(iteration-1)+num_mis)/iteration
  ########## benchmark: full data
  mse,b,c,se=data_to_stats([complete_data],truth1,truth2,missing_row1,missing_row2,missing_col1,missing_col2)
  
  full_beta_estimate=(full_beta_estimate*(iteration-1)+b[1])/iteration
  full_SE=(full_SE*(iteration-1)+se)/iteration
  full_SD.append(b[1])

  ########## benchmark: partial
  X=complete_case[:,:-1]
  ols = sm.OLS(complete_case[:,-1], sm.add_constant(X[:,[0,1,2]]))
  # ols = sm.OLS(complete_case[:,-1], sm.add_constant(X[:,[0,1,2,3,4]]))
  result=ols.fit()
  c=result.cov_params()
  b=result.params
  
  partial_beta_estimate=(partial_beta_estimate*(iteration-1)+b[1])/iteration
  partial_SE=(partial_SE*(iteration-1)+np.sqrt(c[1][1]))/iteration
  partial_SD.append(b[1])

  ########## benchmark: naive imputation
  naive_impute = sample.copy()
  mis1 = naive_impute[:,missing_col1==True]
  mis1[missing_row1,:]=np.mean(mis1[missing_row1==0,:], axis=0)
  mis2 = naive_impute[:,missing_col2==True]
  mis2[missing_row2,:]=np.mean(mis2[missing_row2==0,:], axis=0)
  naive_impute[:,missing_col1==True] = mis1
  naive_impute[:,missing_col2==True] = mis2

  mse,b,c,se=data_to_stats([naive_impute],truth1,truth2,missing_row1,missing_row2,missing_col1,missing_col2)
  
  naive_imp_beta_estimate=(naive_imp_beta_estimate*(iteration-1)+b[1])/iteration
  naive_imp_SE=(naive_imp_SE*(iteration-1)+se)/iteration
  naive_imp_SD.append(b[1])
  naive_imp_MSE=(naive_imp_MSE*(iteration-1)+mse)/iteration

print('full_MSE: NA ', 'full_estimate ', full_beta_estimate, 'full_SE: ',full_SE)
print('partial_MSE: NA ', 'partial_estimate ', partial_beta_estimate, 'partial_SE: ',partial_SE)
print('naive_imp_MSE: ', naive_imp_MSE, 'naive_imp_estimate ', naive_imp_beta_estimate, 'naive_imp_SE: ',naive_imp_SE)
full_StD=np.std(full_SD, dtype=np.float64)
partial_StD=np.std(partial_SD, dtype=np.float64)
naive_imp_StD=np.std(naive_imp_SD, dtype=np.float64)
print('full_StD: ', full_StD)
print('partial_StD: ', partial_StD)
print('naive_imp_StD: ', naive_imp_StD)

# missing rate
print('num_of_missing_row: ',num_of_missing_row)
print('num_of_missing_row1: ',num_of_missing_row1)
print('num_of_missing_row2: ',num_of_missing_row2)

print('n:', n, ' p: ',p, ' Monte Carlo:',Monte_Carlo)
print('a0=',a0,' a1=', a1,' a2=',a2,' a3=',a3,' a4=',a4,' a5=',a5, 'q=',q)

# ***GAIN***

In [ ]:
from gain import gain
from utils import rmse_loss

Model_2_SD=[]
Model_2_SE_list=[]
Model_2_SE=0
Model_2_estimate=0
Model_2_MSE=0
Model_2_time=0

a0=-1
a1=-3
a2=3
a3=-1
a4=-3
a5=2
Monte_Carlo =50

for iteration in range(1,1+Monte_Carlo):
  sample,missing_row1,missing_row2,missing_col1,missing_col2,y=data_G_linear(n=n,p=p,data=adni_data,seed=iteration,a0=a0,a1=a1,a2=a2,a3=a3,a4=a4,a5=a5)
  # print(sum(missing_row))
  truth1=sample[missing_row1==True,:][:,missing_col1==True].copy()
  truth2=sample[missing_row2==True,:][:,missing_col2==True].copy()
  
  s=time.time()
  
  mis1 = sample[:,missing_col1==True].copy()
  mis2 = sample[:,missing_col2==True].copy()
  mis1[missing_row1==True,:] = np.nan
  mis2[missing_row2==True,:] = np.nan
  sample[:,missing_col1==True]=mis1
  sample[:,missing_col2==True]=mis2
  rawdata = sample.copy()

  gain_parameters = {'batch_size': 256,
                    'hint_rate': 0.9,
                    'alpha': 100.0,
                    'iterations': 5000}
  imputed_data_x = gain(rawdata.copy(), gain_parameters)

  mse,b,c,se=data_to_stats([imputed_data_x],truth1,truth2,missing_row1,missing_row2,missing_col1,missing_col2)
  print('Model_2 cost ',time.time()-s,' sec \n','MSE:', mse,'; Model_2_estimate: ',b[1])
  Model_2_SE_list.append(se)
  Model_2_SE=(Model_2_SE*(iteration-1)+se)/iteration
  Model_2_SD.append(b[1])
  Model_2_MSE=(Model_2_MSE*(iteration-1)+mse)/iteration
  Model_2_estimate=(Model_2_estimate*(iteration-1)+b[1])/iteration
  Model_2_time = (Model_2_time*(iteration-1)+time.time()-s)/iteration

  if iteration%5==0:
    print('iteration: ',iteration, 'Model_2_MSE: ',Model_2_MSE,'Model_2_estimate: ',Model_2_estimate,'Model_2_SE: ',Model_2_SE)
    
    Model_2_StD=np.std(Model_2_SD, dtype=np.float64)
    print('Model_2_StD: ',Model_2_StD)
    Model_2_SE_median=statistics.median(Model_2_SE_list)
    print('Model_2 average cost:', Model_2_time)

print('n:', n, ' p: ',p, ' Monte Carlo:', Monte_Carlo)
print('a0=',a0,' a1=', a1,' a2=',a2,' a3=',a3,' a4=',a4,' a5=',a5)

Instructions for updating:
non-resource variables are not supported in the long term


100%|██████████| 5000/5000 [02:45<00:00, 30.26it/s]


Model_2 cost  169.07143878936768  sec 
 MSE: 1.2490769020804864 ; Model_2_estimate:  0.011254740462352626


100%|██████████| 5000/5000 [02:52<00:00, 29.00it/s]


Model_2 cost  173.55925488471985  sec 
 MSE: 1.176442556518863 ; Model_2_estimate:  0.01871784341186143


...

Model_2 cost  180.85665035247803  sec 
 MSE: 1.324056614262477 ; Model_2_estimate:  0.012028211540923804


100%|██████████| 5000/5000 [03:17<00:00, 25.34it/s]


Model_2 cost  215.95159339904785  sec 
 MSE: 1.1742045957847167 ; Model_2_estimate:  0.03448277545939132


100%|██████████| 5000/5000 [02:48<00:00, 29.75it/s]


Model_2 cost  185.44930958747864  sec 
 MSE: 1.220940598113082 ; Model_2_estimate:  0.011584744859672013
iteration:  50 Model_2_MSE:  1.2415066192499424 Model_2_estimate:  0.018786042245526536 Model_2_SE:  0.010464795100259662
Model_2_StD:  0.011477764689112553
Model_2 average cost: 172.27069018363952
n: 649  p:  1500  Monte Carlo: 50
a0= -1  a1= -3  a2= 3  a3= -1  a4= -3  a5= 2


In [ ]:
### MAR ###
iteration:  50 Model_2_MSE:  1.2678542839526705 Model_2_estimate:  0.01871754528667682 Model_2_SE:  0.010535515787772485
Model_2_StD:  0.011843121959576243
Model_2 average cost: 187.677102560997
n: 649  p:  1000  Monte Carlo: 50
a0= -1  a1= -3  a2= 3  a3= -1  a4= -3  a5= 2

iteration:  50 Model_2_MSE:  1.2415066192499424 Model_2_estimate:  0.018786042245526536 Model_2_SE:  0.010464795100259662
Model_2_StD:  0.011477764689112553
Model_2 average cost: 172.27069018363952
n: 649  p:  1500  Monte Carlo: 50
a0= -1  a1= -3  a2= 3  a3= -1  a4= -3  a5= 2

In [ ]:
#############  MNAR  ###################
iteration:  70 Model_2_MSE:  1.2646627567853355 Model_2_estimate:  0.017924419765486255 Model_2_SE:  0.009299124480622537
Model_2_StD:  0.013372046208084708
Model_2 average cost: 84.4776154756546
n: 649  p:  1000  Monte Carlo: 70
a0= -1  a1= -3  a2= 3  a3= -1  a4= -3  a5= 2

# ***optimal transport***

In [ ]:
pip install geomloss

In [ ]:
from imputers import *
Model_2_SD=[]
Model_2_SE_list=[]
Model_2_SE=0
Model_2_MSE=0
Model_2_estimate=0
Model_2_time=0

a0=-1
a1=-3
a2=3
a3=-1
a4=-3
a5=2

Monte_Carlo =50

for iteration in range(1,1+Monte_Carlo):
  sample,missing_row1,missing_row2,missing_col1,missing_col2,y=data_G_linear(n=n,p=p,data=adni_data,seed=iteration,a0=a0,a1=a1,a2=a2,a3=a3,a4=a4,a5=a5)
  # print(sum(missing_row))
  truth1=sample[missing_row1==True,:][:,missing_col1==True].copy()
  truth2=sample[missing_row2==True,:][:,missing_col2==True].copy()
  
  s=time.time()
  
  mis1 = sample[:,missing_col1==True].copy()
  mis2 = sample[:,missing_col2==True].copy()
  mis1[missing_row1==True,:] = np.nan
  mis2[missing_row2==True,:] = np.nan
  sample[:,missing_col1==True]=mis1
  sample[:,missing_col2==True]=mis2
  rawdata = torch.from_numpy(sample)

  alist=[]
  sk_imputer = OTimputer()
  for i in range(1):
    sk_imp = sk_imputer.fit_transform(rawdata)
    data = sk_imp.detach().cpu().numpy()     
    alist.append(data)

  mse,b,c,se=data_to_stats(alist,truth1,truth2,missing_row1,missing_row2,missing_col1,missing_col2)

  print('Model_2 cost ',time.time()-s,' sec \n','MSE:', mse,'; Estimate: ',b[1])
  Model_2_SE_list.append(se)
  Model_2_SE=(Model_2_SE*(iteration-1)+se)/iteration
  Model_2_SD.append(b[1])
  Model_2_MSE=(Model_2_MSE*(iteration-1)+mse)/iteration
  Model_2_estimate=(Model_2_estimate*(iteration-1)+b[1])/iteration
  Model_2_time = (Model_2_time*(iteration-1)+time.time()-s)/iteration

  if iteration%5==0:
    print('iteration: ',iteration, 'Model_2_MSE: ',Model_2_MSE,'Model_2_estimate: ',Model_2_estimate,'Model_2_SE: ',Model_2_SE)
    
    Model_2_StD=np.std(Model_2_SD, dtype=np.float64)
    print('Model_2_StD: ',Model_2_StD)
    # Model_2_SE_median=statistics.median(Model_2_SE_list)
    # print('Model_2_SE_median: ',Model_2_SE_median)
    print('Model_2 average cost:', Model_2_time)

    print('n:', n, ' p: ',p, ' Monte Carlo:',Monte_Carlo, 'q:',q)
    print('a0=',a0,' a1=', a1,' a2=',a2,' a3=',a3,' a4=',a4,' a5=',a5)

# ***SoftImpute***

In [ ]:
from softimpute import *
from utils import *

Model_2_SD=[]
Model_2_SE_list=[]
Model_2_SE=0
Model_2_MSE=0
Model_2_estimate=0
Model_2_time=0

a0=-1
a1=-3
a2=3
a3=-1
a4=-3
a5=2
Monte_Carlo = 30

grid_len = 5
maxit = 500
thresh = 1e-4

for iteration in range(1,1+Monte_Carlo):
  sample,missing_row1,missing_row2,missing_col1,missing_col2,y=data_G_linear(n=n,p=p,data=adni_data,seed=iteration,a0=a0,a1=a1,a2=a2,a3=a3,a4=a4,a5=a5)
  # print(sum(missing_row))
  truth1=sample[missing_row1==True,:][:,missing_col1==True].copy()
  truth2=sample[missing_row2==True,:][:,missing_col2==True].copy()
  
  
  mis1 = sample[:,missing_col1==True].copy()
  mis2 = sample[:,missing_col2==True].copy()
  mis1[missing_row1==True,:] = np.nan
  mis2[missing_row2==True,:] = np.nan
  sample[:,missing_col1==True]=mis1
  sample[:,missing_col2==True]=mis2
  rawdata = sample.copy()
  
  cv_error, grid_lambda = cv_softimpute(rawdata.copy(), grid_len = grid_len, maxit = 50, thresh=1e-3)
  s=time.time()
  U_thresh, imp = softimpute(rawdata.copy(), grid_lambda[np.argmin(cv_error)], maxit = maxit, thresh=thresh)
  alist=[imp]

  mse,b,c,se=data_to_stats(alist,truth1,truth2,missing_row1,missing_row2,missing_col1,missing_col2)

  print('Model_2 cost ',time.time()-s,' sec \n','MSE:', mse,'; Estimate: ',b[1])
  Model_2_SE_list.append(se)
  Model_2_SE=(Model_2_SE*(iteration-1)+se)/iteration
  Model_2_SD.append(b[1])
  Model_2_MSE=(Model_2_MSE*(iteration-1)+mse)/iteration
  Model_2_estimate=(Model_2_estimate*(iteration-1)+b[1])/iteration
  Model_2_time = (Model_2_time*(iteration-1)+time.time()-s)/iteration

  if iteration%5==0:
    print('iteration: ',iteration, 'Model_2_MSE: ',Model_2_MSE,'Model_2_estimate: ',Model_2_estimate,'Model_2_SE: ',Model_2_SE)
    
    Model_2_StD=np.std(Model_2_SD, dtype=np.float64)
    print('Model_2_StD: ',Model_2_StD)
    Model_2_SE_median=statistics.median(Model_2_SE_list)
    print('Model_2_SE_median: ',Model_2_SE_median)
    print('Model_2 average cost:', Model_2_time)

print('n:', n, ' p: ',p, ' Monte Carlo:',Monte_Carlo)
print('a0=',a0,' a1=', a1,' a2=',a2,' a3=',a3,' a4=',a4,' a5=',a5)
print('grid_len=',grid_len, 'maxit=', maxit, 'thresh=',thresh)

# **MIGAN2**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import grad
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import WeightedRandomSampler
from torchvision import datasets, transforms
import torch.nn.functional as F

In [ ]:
class Generate_data_model(Dataset):   
  def __init__(self, array, transform=None):  
    self.array = array
    self.transform = transform

  def __len__(self):
    return len(self.array)

  def __getitem__(self, index):
    return self.array[index]

class Generate_data_model_2(Dataset):   
  def __init__(self, array, mask, transform=None):  
    self.array = array
    self.mask = mask             
    self.transform = transform

  def __len__(self):
    return len(self.array)

  def __getitem__(self, index):
    return self.array[index], self.mask

class Imputer_model_2(nn.Module):
  def __init__(self, n1=251, n2=200, n3=100, n4=100):
    super().__init__()
    self.fc = nn.Sequential(
        nn.Linear(n1, n2),
        # nn.ReLU(),
        nn.Tanh(),
        nn.Linear(n2, n3),
        # nn.ReLU(),
        nn.Tanh(),
        nn.Linear(n3, n4),
    )
  def forward(self, data, mask, noise):
    net = data * (1 - mask) + noise * mask
    net = net.view(data.shape[0], -1)
    net = self.fc(net.float())
    net = net.view(data.shape[0], -1)
    return net

class Critic_model_2(nn.Module):
  def __init__(self, n1=251, n2=200, n3=50, n4=1):  #n1=251, n2=100, n3=20, n4=1
    super().__init__()
    self.fc = nn.Sequential(
        nn.Linear(n1, n2),
        nn.ReLU(),   
        # nn.Tanh(),
        nn.Linear(n2, n3),
        nn.ReLU(),   
        # nn.Tanh(),
        nn.Linear(n3, n4),
    )
  def forward(self, x):
    out = self.fc(x)
    return out


class CriticUpdater_model:
  def __init__(self, critic, critic_optimizer, batch_size=8, gp_lambda=10, device='cuda:0'):
    self.critic = critic
    self.critic_optimizer = critic_optimizer
    self.gp_lambda = gp_lambda
    # Interpolation coefficient
    self.eps = torch.empty(batch_size, 1,device=device)
    # For computing the gradient penalty
    self.ones = torch.ones(batch_size, 1).to(device)

  def __call__(self, real, fake, device='cuda:0'):
    real = real.detach()
    fake = fake.detach()
    self.critic.zero_grad()
    self.eps.uniform_(0, 1)
    interp = (self.eps * real + (1 - self.eps) * fake).to(device).requires_grad_()
    grad_d = grad(outputs=self.critic(interp.float()), inputs=interp, grad_outputs=self.ones,create_graph=True)[0]  #grad_outputs=self.ones,create_graph=True
    grad_d = grad_d.view(real.shape[0], -1)
    grad_penalty = ((grad_d.norm(dim=1) - 1)**2).mean() * self.gp_lambda
    w_dist = self.critic(fake.float()).mean() - self.critic(real.float()).mean()
    loss = w_dist + grad_penalty
    loss.backward()
    self.critic_optimizer.step()

class CriticUpdater_model_2:
  def __init__(self, critic, critic_optimizer, batch_size=8, gp_lambda=10, device='cuda:0'):
    self.critic = critic
    self.critic_optimizer = critic_optimizer
    self.gp_lambda = gp_lambda
    # Interpolation coefficient
    self.eps = torch.empty(batch_size, 1,device=device)
    # For computing the gradient penalty
    self.ones = torch.ones(batch_size, 1).to(device)

  def __call__(self, real, fake1, fake2, fake3, device='cuda:0'):
    real = real.detach()
    fake1 = fake1.detach()
    fake2 = fake2.detach()
    fake3 = fake3.detach()
    self.critic.zero_grad()
    self.eps.uniform_(0, 1)
    interp1 = (self.eps * real + (1 - self.eps) * fake1).to(device).requires_grad_()
    self.eps.uniform_(0, 1)
    interp2 = (self.eps * real + (1 - self.eps) * fake2).to(device).requires_grad_()
    self.eps.uniform_(0, 1)
    interp3 = (self.eps * real + (1 - self.eps) * fake3).to(device).requires_grad_()
    grad_d1 = grad(outputs=self.critic(interp1.float()), inputs=interp1, grad_outputs=self.ones,create_graph=True)[0].view(real.shape[0], -1)  #grad_outputs=self.ones,create_graph=True
    grad_d2 = grad(outputs=self.critic(interp2.float()), inputs=interp2, grad_outputs=self.ones,create_graph=True)[0].view(real.shape[0], -1)
    grad_d3 = grad(outputs=self.critic(interp3.float()), inputs=interp3, grad_outputs=self.ones,create_graph=True)[0].view(real.shape[0], -1)
    grad_penalty1 = ((grad_d1.norm(dim=1) - 1)**2).mean() * self.gp_lambda
    grad_penalty2 = ((grad_d2.norm(dim=1) - 1)**2).mean() * self.gp_lambda
    grad_penalty3 = ((grad_d3.norm(dim=1) - 1)**2).mean() * self.gp_lambda
    w_dist1 = self.critic(fake1.float()).mean() - self.critic(real.float()).mean()
    w_dist2 = self.critic(fake2.float()).mean() - self.critic(real.float()).mean()
    w_dist3 = self.critic(fake3.float()).mean() - self.critic(real.float()).mean()
    loss = w_dist1 + w_dist2 + w_dist3 + grad_penalty1 + grad_penalty2 + grad_penalty3 
    loss.backward()
    self.critic_optimizer.step()

def mlp_MIGAN2(missing_data, missing_row1,missing_row2,missing_col1,missing_col2, n=200, p_f=250, device='cuda:0', Batch_size=16, ilr=1e-3, clr=1e-3, gamma=0.2, weight=0.1, gp_lambda=10, epochs=300, MI_seed=1):
  rawdata = missing_data.copy()
  rawdata_inc1 = rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2)), :].copy()
  rawdata_inc2 = rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1)), :].copy()
  rawdata_inc3 = rawdata[np.logical_and(missing_row1,missing_row2), :].copy()
  cc = torch.from_numpy(rawdata.copy()[np.logical_or(missing_row1,missing_row2)==0, :])

  cc_data = Generate_data_model(cc)
  cc_data_pat1 =  Generate_data_model_2(cc, missing_col1)
  cc_data_pat2 =  Generate_data_model_2(cc, missing_col2)
  cc_data_pat3 =  Generate_data_model_2(cc, np.logical_or(missing_col1,missing_col2))

  cc_data_loader = DataLoader(cc_data, batch_size=Batch_size, shuffle=True)
  cc_data_loader1 = DataLoader(cc_data_pat1, batch_size=Batch_size, shuffle=True)
  cc_data_loader2 = DataLoader(cc_data_pat2, batch_size=Batch_size, shuffle=True)
  cc_data_loader3 = DataLoader(cc_data_pat3, batch_size=Batch_size, shuffle=True)

  imputer1 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col1)).to(device)
  imputer2 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col2)).to(device)
  imputer3 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(np.logical_or(missing_col1,missing_col2))).to(device)  

  critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)    
  imputer_params = list(imputer1.parameters()) + list(imputer2.parameters()) + list(imputer3.parameters())
  imputer_optimizer = optim.Adam(imputer_params, lr=ilr, betas=(.5, .9))
  critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
  scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
  scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
  mse_loss = torch.nn.MSELoss(reduction='mean')
  l1_loss = nn.L1Loss(reduction='mean')

  critic_updates = 0
  n_critic = 5

  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
     
      cc_1, mask1 = next(iter(cc_data_loader1))
      cc_1 = cc_1[0:batch_size].to(device).float()
      mask1 = mask1[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data1 = imputer1(cc_1, mask1, impu_noise)
      cc_hat1 = cc_1.clone()
      cc_hat1[:,missing_col1] = imputed_data1

      cc_2, mask2 = next(iter(cc_data_loader2))
      cc_2 = cc_2[0:batch_size].to(device).float()
      mask2 = mask2[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data2 = imputer2(cc_2, mask2, impu_noise)
      cc_hat2 = cc_2.clone()
      cc_hat2[:,missing_col2] = imputed_data2

      cc_3, mask3 = next(iter(cc_data_loader3))
      cc_3 = cc_3[0:batch_size].to(device).float()
      mask3 = mask3[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data3 = imputer3(cc_3, mask3, impu_noise)
      cc_hat3 = cc_3.clone()
      cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3
      
      update_critic = CriticUpdater_model_2(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat1, cc_hat2, cc_hat3)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0

        impu_noise.normal_(mean=0,std=1)
        imputed_data1 = imputer1(cc_1, mask1, impu_noise)
        cc_hat1 = cc_1.clone()
        cc_hat1[:,missing_col1] = imputed_data1
        reconstruction_loss += l1_loss(imputed_data1, cc_1[:,missing_col1].float())
        
        impu_noise.normal_(mean=0,std=1)
        imputed_data2 = imputer2(cc_2, mask2, impu_noise)
        cc_hat2 = cc_2.clone()
        cc_hat2[:,missing_col2] = imputed_data2
        reconstruction_loss += l1_loss(imputed_data2, cc_2[:,missing_col2].float())

        impu_noise.normal_(mean=0,std=1)
        imputed_data3 = imputer3(cc_3, mask3, impu_noise)
        cc_hat3 = cc_3.clone()
        cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3
        reconstruction_loss += l1_loss(imputed_data3, cc_3[:,np.logical_or(missing_col1,missing_col2)].float())

        loss = -critic(cc_hat1).mean()-critic(cc_hat2).mean()-critic(cc_hat3).mean() + weight*reconstruction_loss
        
        imputer1.zero_grad()
        imputer2.zero_grad()
        imputer3.zero_grad()
        loss.backward()
        imputer_optimizer.step()

    scheduler1.step()
    scheduler2.step()

  alist = []
  with torch.no_grad():
    imputer1.eval()
    imputer2.eval()
    imputer3.eval()
    inc_data_pat1 = torch.from_numpy(rawdata_inc1).to(device)
    inc_data_pat2 = torch.from_numpy(rawdata_inc2).to(device)
    inc_data_pat3 = torch.from_numpy(rawdata_inc3).to(device)
    mask1 = torch.from_numpy(missing_col1).to(device)
    mask2 = torch.from_numpy(missing_col2).to(device)
    mask3 = torch.from_numpy(np.logical_and(missing_col1,missing_col2)).to(device)
    for i in range(1):
      torch.manual_seed(MI_seed)
      impu_noise = torch.empty(sum(np.logical_and(missing_row1,np.logical_not(missing_row2))), p_f+1, device=device)
      imputed_data = imputer1(inc_data_pat1.float(), mask1.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc1[:,missing_col1] = imputed_data.cpu().numpy()
      # print(sum(np.isnan(imputed_data.cpu().numpy())))
      rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2))] = rawdata_inc1.copy()
      
      impu_noise = torch.empty(sum(np.logical_and(missing_row2,np.logical_not(missing_row1))), p_f+1, device=device)
      imputed_data = imputer2(inc_data_pat2.float(), mask2.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc2[:,missing_col2] = imputed_data.cpu().numpy()
      # print(sum(np.isnan(imputed_data.cpu().numpy())))
      rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1))] = rawdata_inc2.copy()

      impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2)), p_f+1, device=device)
      imputed_data = imputer3(inc_data_pat3.float(), mask3.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data.cpu().numpy()
      # print(sum(np.isnan(imputed_data.cpu().numpy())))
      rawdata[np.logical_and(missing_row1, missing_row2)] = rawdata_inc3.copy()

      alist.append(rawdata.copy())
  return alist

## iterative approach, column pattern 

In [ ]:
def mlp_MIGAN2_iterative(missing_data, missing_row1,missing_row2,missing_col1,missing_col2, n=200, p_f=250, device='cuda:0', Batch_size=16, ilr=1e-3, clr=1e-3, gamma=0.2, weight=0.1, gp_lambda=10, epochs=300, MI=10):
  rawdata = missing_data.copy()
  rawdata_inc1 = rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2)), :].copy()
  rawdata_inc2 = rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1)), :].copy()
  rawdata_inc3 = rawdata[np.logical_and(missing_row1,missing_row2), :].copy()
  cc = torch.from_numpy(rawdata.copy()[np.logical_or(missing_row1,missing_row2)==0, :])

  cc_data = Generate_data_model(cc)
  cc_data_pat1 =  Generate_data_model_2(cc, missing_col1)
  cc_data_pat2 =  Generate_data_model_2(cc, missing_col2)
  cc_data_pat3 =  Generate_data_model_2(cc, np.logical_or(missing_col1,missing_col2))

  cc_data_loader = DataLoader(cc_data, batch_size=Batch_size, shuffle=True)
  cc_data_loader1 = DataLoader(cc_data_pat1, batch_size=Batch_size, shuffle=True)
  cc_data_loader2 = DataLoader(cc_data_pat2, batch_size=Batch_size, shuffle=True)
  cc_data_loader3 = DataLoader(cc_data_pat3, batch_size=Batch_size, shuffle=True)
  
  imputer1 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col1)).to(device)
  imputer2 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col2)).to(device)
  imputer3 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(np.logical_or(missing_col1,missing_col2))).to(device)  

  critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)    
  imputer_params = list(imputer1.parameters()) + list(imputer2.parameters()) + list(imputer3.parameters())
  imputer_optimizer = optim.Adam(imputer_params, lr=ilr, betas=(.5, .9))
  critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
  scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
  scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
  mse_loss = torch.nn.MSELoss(reduction='mean')
  l1_loss = nn.L1Loss(reduction='mean')

  critic_updates = 0
  n_critic = 5

  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
     
      cc_1, mask1 = next(iter(cc_data_loader1))
      cc_1 = cc_1[0:batch_size].to(device).float()
      mask1 = mask1[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data1 = imputer1(cc_1, mask1, impu_noise)
      cc_hat1 = cc_1.clone()
      cc_hat1[:,missing_col1] = imputed_data1

      cc_2, mask2 = next(iter(cc_data_loader2))
      cc_2 = cc_2[0:batch_size].to(device).float()
      mask2 = mask2[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data2 = imputer2(cc_2, mask2, impu_noise)
      cc_hat2 = cc_2.clone()
      cc_hat2[:,missing_col2] = imputed_data2

      cc_3, mask3 = next(iter(cc_data_loader3))
      cc_3 = cc_3[0:batch_size].to(device).float()
      mask3 = mask3[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data3 = imputer3(cc_3, mask3, impu_noise)
      cc_hat3 = cc_3.clone()
      cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3
      
      update_critic = CriticUpdater_model_2(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat1, cc_hat2, cc_hat3)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0

        impu_noise.normal_(mean=0,std=1)
        imputed_data1 = imputer1(cc_1, mask1, impu_noise)
        cc_hat1 = cc_1.clone()
        cc_hat1[:,missing_col1] = imputed_data1
        reconstruction_loss += l1_loss(imputed_data1, cc_1[:,missing_col1].float())
        
        impu_noise.normal_(mean=0,std=1)
        imputed_data2 = imputer2(cc_2, mask2, impu_noise)
        cc_hat2 = cc_2.clone()
        cc_hat2[:,missing_col2] = imputed_data2
        reconstruction_loss += l1_loss(imputed_data2, cc_2[:,missing_col2].float())

        impu_noise.normal_(mean=0,std=1)
        imputed_data3 = imputer3(cc_3, mask3, impu_noise)
        cc_hat3 = cc_3.clone()
        cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3
        reconstruction_loss += l1_loss(imputed_data3, cc_3[:,np.logical_or(missing_col1,missing_col2)].float())

        loss = -critic(cc_hat1).mean()-critic(cc_hat2).mean()-critic(cc_hat3).mean() + weight*reconstruction_loss
        
        imputer1.zero_grad()
        imputer2.zero_grad()
        imputer3.zero_grad()
        loss.backward()
        imputer_optimizer.step()

    scheduler1.step()
    scheduler2.step()

  with torch.no_grad():
    imputer1.eval()
    imputer2.eval()
    imputer3.eval()
    inc_data_pat1 = torch.from_numpy(rawdata_inc1).to(device)
    inc_data_pat2 = torch.from_numpy(rawdata_inc2).to(device)
    inc_data_pat3 = torch.from_numpy(rawdata_inc3).to(device)
    mask1 = torch.from_numpy(missing_col1).to(device)
    mask2 = torch.from_numpy(missing_col2).to(device)
    mask3 = torch.from_numpy(np.logical_and(missing_col1,missing_col2)).to(device)
    for i in range(1):
      impu_noise = torch.empty(sum(np.logical_and(missing_row1,np.logical_not(missing_row2))), p_f+1, device=device)
      imputed_data = imputer1(inc_data_pat1.float(), mask1.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc1[:,missing_col1] = imputed_data.cpu().numpy()
      rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2))] = rawdata_inc1.copy()
      
      impu_noise = torch.empty(sum(np.logical_and(missing_row2,np.logical_not(missing_row1))), p_f+1, device=device)
      imputed_data = imputer2(inc_data_pat2.float(), mask2.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc2[:,missing_col2] = imputed_data.cpu().numpy()
      rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1))] = rawdata_inc2.copy()

      impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2)), p_f+1, device=device)
      imputed_data = imputer3(inc_data_pat3.float(), mask3.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data.cpu().numpy()
      rawdata[np.logical_and(missing_row1, missing_row2)] = rawdata_inc3.copy()
  
  alist = []
  for j in range(1+MI):
    rawdata_pat1 = rawdata[missing_row1, :].copy()
    train_pat1 = rawdata[np.logical_not(missing_row1), :].copy()
    train_pat1 =  Generate_data_model_2(train_pat1, missing_col1)
    train_pat1_loader1 = DataLoader(train_pat1, batch_size=Batch_size, shuffle=True)
    train_pat1_loader2 = DataLoader(train_pat1, batch_size=Batch_size, shuffle=True)
    
    imputer1 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col1)).to(device) 
    critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)    
    imputer_optimizer = optim.Adam(imputer1.parameters(), lr=ilr, betas=(.5, .9))
    critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
    scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
    scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
    
    critic_updates = 0
    for epoch in range(1,1+epochs):
      for cc, mask in train_pat1_loader1:      
        cc = cc.to(device).float()                           
        batch_size = cc.shape[0]
        impu_noise = torch.empty(batch_size, p_f+1, device=device)
      
        cc_1, mask1 = next(iter(train_pat1_loader2))
        cc_1 = cc_1[0:batch_size].to(device).float()
        mask1 = mask1[0:batch_size].to(device).float()
        impu_noise.normal_(mean=0,std=1)
        imputed_data1 = imputer1(cc_1, mask1, impu_noise)
        cc_hat1 = cc_1.clone()
        cc_hat1[:,missing_col1] = imputed_data1
        update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
        update_critic(cc, cc_hat1)
        critic_updates += 1

        if critic_updates == n_critic:
          critic_updates = 0
          reconstruction_loss = 0

          impu_noise.normal_(mean=0,std=1)
          imputed_data1 = imputer1(cc_1, mask1, impu_noise)
          cc_hat1 = cc_1.clone()
          cc_hat1[:,missing_col1] = imputed_data1
          reconstruction_loss += l1_loss(imputed_data1, cc_1[:,missing_col1].float())
          loss = -critic(cc_hat1).mean() + weight*reconstruction_loss
          imputer1.zero_grad()
          loss.backward()
          imputer_optimizer.step()
      scheduler1.step()
      scheduler2.step()
    
    with torch.no_grad():
      imputer1.eval()
      inc_data_pat1 = torch.from_numpy(rawdata_pat1).to(device)
      mask1 = torch.from_numpy(missing_col1).to(device)
      for i in range(1):
        torch.manual_seed(j)
        impu_noise = torch.empty(sum(missing_row1), p_f+1, device=device)
        imputed_data = imputer1(inc_data_pat1.float(), mask1.float(), impu_noise.normal_(mean=0,std=1)) 
        rawdata_pat1[:,missing_col1] = imputed_data.cpu().numpy()
        rawdata[missing_row1] = rawdata_pat1.copy()

    rawdata_pat2 = rawdata[missing_row2, :].copy()
    train_pat2 = rawdata[np.logical_not(missing_row2), :].copy()
    train_pat2 =  Generate_data_model_2(train_pat2, missing_col2)
    
    train_pat2_loader1 = DataLoader(train_pat2, batch_size=Batch_size, shuffle=True)
    train_pat2_loader2 = DataLoader(train_pat2, batch_size=Batch_size, shuffle=True)
    imputer2 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col2)).to(device) 
    critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
    imputer_optimizer = optim.Adam(imputer2.parameters(), lr=ilr, betas=(.5, .9))
    critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
    scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
    scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
    critic_updates = 0

    for epoch in range(1,1+epochs):
      for cc, mask in train_pat2_loader1:      
        cc = cc.to(device).float()                           
        batch_size = cc.shape[0]
        impu_noise = torch.empty(batch_size, p_f+1, device=device)
      
        cc_2, mask2 = next(iter(train_pat2_loader2))
        cc_2 = cc_2[0:batch_size].to(device).float()
        mask2 = mask2[0:batch_size].to(device).float()
        impu_noise.normal_(mean=0,std=1)
        imputed_data2 = imputer2(cc_2, mask2, impu_noise)
        cc_hat2 = cc_2.clone()
        cc_hat2[:,missing_col2] = imputed_data2

        update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
        update_critic(cc, cc_hat2)
        critic_updates += 1

        if critic_updates == n_critic:
          critic_updates = 0
          reconstruction_loss = 0
          
          impu_noise.normal_(mean=0,std=1)
          imputed_data2 = imputer2(cc_2, mask2, impu_noise)
          cc_hat2 = cc_2.clone()
          cc_hat2[:,missing_col2] = imputed_data2
          reconstruction_loss += l1_loss(imputed_data2, cc_2[:,missing_col2].float())
          loss = -critic(cc_hat2).mean() + weight*reconstruction_loss
          imputer2.zero_grad()
          loss.backward()
          imputer_optimizer.step()
      scheduler1.step()
      scheduler2.step()

    with torch.no_grad():
      imputer2.eval()
      inc_data_pat2 = torch.from_numpy(rawdata_pat2).to(device)
      mask2 = torch.from_numpy(missing_col2).to(device)
      for i in range(1):
        torch.manual_seed(j)
        impu_noise = torch.empty(sum(missing_row2), p_f+1, device=device)
        imputed_data = imputer2(inc_data_pat2.float(), mask2.float(), impu_noise.normal_(mean=0,std=1)) 
        rawdata_pat2[:,missing_col2] = imputed_data.cpu().numpy()
        rawdata[missing_row2] = rawdata_pat2.copy()

    if j > 0:
      alist.append(rawdata.copy())
  return alist

In [ ]:
def mlp_MIGAN2_iterative5(missing_data, missing_row1,missing_row2,missing_col1,missing_col2, n=200, p_f=250, device='cuda:0', Batch_size=16, ilr=1e-3, clr=1e-3, gamma=0.2, weight=0.1, gp_lambda=10, epochs=300, MI=10):
  rawdata = missing_data.copy()
  rawdata_pat1 = rawdata[missing_row1, :].copy()
  rawdata_pat2 = rawdata[missing_row2, :].copy()
  
  imputer1 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col1)).to(device) 
  critic1 = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)    
  imputer2 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col2)).to(device) 
  critic2 = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
  
  mse_loss = torch.nn.MSELoss(reduction='mean')
  l1_loss = nn.L1Loss(reduction='mean')

  n_critic = 5
  alist = []
  for j in range(3+MI):
    if j==0:
      train_pat1 = rawdata[np.logical_or(missing_row1,missing_row2)==0, :].copy()
      train_pat1 = Generate_data_model_2(train_pat1, missing_col1)
      train_pat1_loader1 = DataLoader(train_pat1, batch_size=Batch_size, shuffle=True)
      train_pat1_loader2 = DataLoader(train_pat1, batch_size=Batch_size, shuffle=True)
    if j>0:
      train_pat1 = rawdata[np.logical_not(missing_row1), :].copy()
      train_pat1 =  Generate_data_model_2(train_pat1, missing_col1)
      train_pat1_loader1 = DataLoader(train_pat1, batch_size=Batch_size, shuffle=True)
      train_pat1_loader2 = DataLoader(train_pat1, batch_size=Batch_size, shuffle=True)
    
    imputer_optimizer1 = optim.Adam(imputer1.parameters(), lr=ilr, betas=(.5, .9))
    critic_optimizer1 = optim.Adam(critic1.parameters(), lr=clr, betas=(.5, .9))
    scheduler1_1 = optim.lr_scheduler.StepLR(imputer_optimizer1, step_size=10, gamma=gamma)
    scheduler1_2 = optim.lr_scheduler.StepLR(critic_optimizer1, step_size=10, gamma=gamma)
    critic_updates = 0
    imputer1.train()
    for epoch in range(1,1+epochs):
      for cc, mask in train_pat1_loader1:      
        cc = cc.to(device).float()                           
        batch_size = cc.shape[0]
        impu_noise = torch.empty(batch_size, p_f+1, device=device)
      
        cc_1, mask1 = next(iter(train_pat1_loader2))
        cc_1 = cc_1[0:batch_size].to(device).float()
        mask1 = mask1[0:batch_size].to(device).float()
        impu_noise.normal_(mean=0,std=1)
        imputed_data1 = imputer1(cc_1, mask1, impu_noise)
        cc_hat1 = cc_1.clone()
        cc_hat1[:,missing_col1] = imputed_data1
        update_critic = CriticUpdater_model(critic1, critic_optimizer1, batch_size, device=device, gp_lambda=gp_lambda)
        update_critic(cc, cc_hat1)
        critic_updates += 1

        if critic_updates == n_critic:
          critic_updates = 0
          reconstruction_loss = 0

          impu_noise.normal_(mean=0,std=1)
          imputed_data1 = imputer1(cc_1, mask1, impu_noise)
          cc_hat1 = cc_1.clone()
          cc_hat1[:,missing_col1] = imputed_data1
          reconstruction_loss += l1_loss(imputed_data1, cc_1[:,missing_col1].float())
          loss = -critic1(cc_hat1).mean() + weight*reconstruction_loss
          imputer1.zero_grad()
          loss.backward()
          imputer_optimizer1.step()
      scheduler1_1.step()
      scheduler1_2.step()
    
    with torch.no_grad():
      imputer1.eval()
      inc_data_pat1 = torch.from_numpy(rawdata_pat1).to(device)
      mask1 = torch.from_numpy(missing_col1).to(device)
      for i in range(1):
        torch.manual_seed(j)
        impu_noise = torch.empty(sum(missing_row1), p_f+1, device=device)
        imputed_data = imputer1(inc_data_pat1.float(), mask1.float(), impu_noise.normal_(mean=0,std=1)) 
        rawdata_pat1[:,missing_col1] = imputed_data.cpu().numpy()
        rawdata[missing_row1] = rawdata_pat1.copy()
    
    if j==0:
      train_pat2 = rawdata[np.logical_or(missing_row1,missing_row2)==0, :].copy()
      train_pat2 =  Generate_data_model_2(train_pat2, missing_col2)
      train_pat2_loader1 = DataLoader(train_pat2, batch_size=Batch_size, shuffle=True)
      train_pat2_loader2 = DataLoader(train_pat2, batch_size=Batch_size, shuffle=True)
    if j>0:
      train_pat2 = rawdata[np.logical_not(missing_row2), :].copy()
      train_pat2 =  Generate_data_model_2(train_pat2, missing_col2)
      train_pat2_loader1 = DataLoader(train_pat2, batch_size=Batch_size, shuffle=True)
      train_pat2_loader2 = DataLoader(train_pat2, batch_size=Batch_size, shuffle=True)
    
    imputer_optimizer2 = optim.Adam(imputer2.parameters(), lr=ilr, betas=(.5, .9))
    critic_optimizer2 = optim.Adam(critic2.parameters(), lr=clr, betas=(.5, .9))
    scheduler2_1 = optim.lr_scheduler.StepLR(imputer_optimizer2, step_size=10, gamma=gamma)
    scheduler2_2 = optim.lr_scheduler.StepLR(critic_optimizer2, step_size=10, gamma=gamma)
    critic_updates = 0
    imputer2.train()
    for epoch in range(1,1+epochs):
      for cc, mask in train_pat2_loader1:      
        cc = cc.to(device).float()                           
        batch_size = cc.shape[0]
        impu_noise = torch.empty(batch_size, p_f+1, device=device)
      
        cc_2, mask2 = next(iter(train_pat2_loader2))
        cc_2 = cc_2[0:batch_size].to(device).float()
        mask2 = mask2[0:batch_size].to(device).float()
        impu_noise.normal_(mean=0,std=1)
        imputed_data2 = imputer2(cc_2, mask2, impu_noise)
        cc_hat2 = cc_2.clone()
        cc_hat2[:,missing_col2] = imputed_data2

        update_critic = CriticUpdater_model(critic2, critic_optimizer2, batch_size, device=device, gp_lambda=gp_lambda)
        update_critic(cc, cc_hat2)
        critic_updates += 1

        if critic_updates == n_critic:
          critic_updates = 0
          reconstruction_loss = 0
          
          impu_noise.normal_(mean=0,std=1)
          imputed_data2 = imputer2(cc_2, mask2, impu_noise)
          cc_hat2 = cc_2.clone()
          cc_hat2[:,missing_col2] = imputed_data2
          reconstruction_loss += l1_loss(imputed_data2, cc_2[:,missing_col2].float())
          loss = -critic2(cc_hat2).mean() + weight*reconstruction_loss
          imputer2.zero_grad()
          loss.backward()
          imputer_optimizer2.step()
      scheduler2_1.step()
      scheduler2_2.step()

    with torch.no_grad():
      imputer2.eval()
      inc_data_pat2 = torch.from_numpy(rawdata_pat2).to(device)
      mask2 = torch.from_numpy(missing_col2).to(device)
      for i in range(1):
        torch.manual_seed(j)
        impu_noise = torch.empty(sum(missing_row2), p_f+1, device=device)
        imputed_data = imputer2(inc_data_pat2.float(), mask2.float(), impu_noise.normal_(mean=0,std=1)) 
        rawdata_pat2[:,missing_col2] = imputed_data.cpu().numpy()
        rawdata[missing_row2] = rawdata_pat2.copy()

    if j > 2:
      alist.append(rawdata.copy())
  return alist

## iterative approach, row pattern, 1

In [ ]:
def mlp_MIGAN2_iterative2(missing_data, missing_row1,missing_row2,missing_col1,missing_col2, n=200, p_f=250, device='cuda:0', Batch_size=16, ilr=1e-3, clr=1e-3, gamma=0.2, weight=0.1, gp_lambda=10, epochs=300, MI=10):
  rawdata = missing_data.copy()
  rawdata_inc1 = rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2)), :].copy()
  rawdata_inc2 = rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1)), :].copy()
  rawdata_inc3 = rawdata[np.logical_and(missing_row1,missing_row2), :].copy()
  cc = torch.from_numpy(rawdata.copy()[np.logical_or(missing_row1,missing_row2)==0, :])

  cc_data = Generate_data_model(cc)
  cc_data_pat1 =  Generate_data_model_2(cc, missing_col1)
  cc_data_pat2 =  Generate_data_model_2(cc, missing_col2)
  cc_data_pat3 =  Generate_data_model_2(cc, np.logical_or(missing_col1,missing_col2))

  cc_data_loader = DataLoader(cc_data, batch_size=Batch_size, shuffle=True)
  cc_data_loader1 = DataLoader(cc_data_pat1, batch_size=Batch_size, shuffle=True)
  cc_data_loader2 = DataLoader(cc_data_pat2, batch_size=Batch_size, shuffle=True)
  cc_data_loader3 = DataLoader(cc_data_pat3, batch_size=Batch_size, shuffle=True)
  
  imputer1 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col1)).to(device)
  imputer2 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col2)).to(device)
  imputer3 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(np.logical_or(missing_col1,missing_col2))).to(device)  

  critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)    
  imputer_params = list(imputer1.parameters()) + list(imputer2.parameters()) + list(imputer3.parameters())
  imputer_optimizer = optim.Adam(imputer_params, lr=ilr, betas=(.5, .9))
  critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
  scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
  scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
  mse_loss = torch.nn.MSELoss(reduction='mean')
  l1_loss = nn.L1Loss(reduction='mean')

  critic_updates = 0
  n_critic = 5

  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
     
      cc_1, mask1 = next(iter(cc_data_loader1))
      cc_1 = cc_1[0:batch_size].to(device).float()
      mask1 = mask1[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data1 = imputer1(cc_1, mask1, impu_noise)
      cc_hat1 = cc_1.clone()
      cc_hat1[:,missing_col1] = imputed_data1

      cc_2, mask2 = next(iter(cc_data_loader2))
      cc_2 = cc_2[0:batch_size].to(device).float()
      mask2 = mask2[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data2 = imputer2(cc_2, mask2, impu_noise)
      cc_hat2 = cc_2.clone()
      cc_hat2[:,missing_col2] = imputed_data2

      cc_3, mask3 = next(iter(cc_data_loader3))
      cc_3 = cc_3[0:batch_size].to(device).float()
      mask3 = mask3[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data3 = imputer3(cc_3, mask3, impu_noise)
      cc_hat3 = cc_3.clone()
      cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3
      
      update_critic = CriticUpdater_model_2(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat1, cc_hat2, cc_hat3)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0

        impu_noise.normal_(mean=0,std=1)
        imputed_data1 = imputer1(cc_1, mask1, impu_noise)
        cc_hat1 = cc_1.clone()
        cc_hat1[:,missing_col1] = imputed_data1
        reconstruction_loss += l1_loss(imputed_data1, cc_1[:,missing_col1].float())
        
        impu_noise.normal_(mean=0,std=1)
        imputed_data2 = imputer2(cc_2, mask2, impu_noise)
        cc_hat2 = cc_2.clone()
        cc_hat2[:,missing_col2] = imputed_data2
        reconstruction_loss += l1_loss(imputed_data2, cc_2[:,missing_col2].float())

        impu_noise.normal_(mean=0,std=1)
        imputed_data3 = imputer3(cc_3, mask3, impu_noise)
        cc_hat3 = cc_3.clone()
        cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3
        reconstruction_loss += l1_loss(imputed_data3, cc_3[:,np.logical_or(missing_col1,missing_col2)].float())

        loss = -critic(cc_hat1).mean()-critic(cc_hat2).mean()-critic(cc_hat3).mean() + weight*reconstruction_loss
        
        imputer1.zero_grad()
        imputer2.zero_grad()
        imputer3.zero_grad()
        loss.backward()
        imputer_optimizer.step()

    scheduler1.step()
    scheduler2.step()

  with torch.no_grad():
    imputer1.eval()
    imputer2.eval()
    imputer3.eval()
    inc_data_pat1 = torch.from_numpy(rawdata_inc1).to(device)
    inc_data_pat2 = torch.from_numpy(rawdata_inc2).to(device)
    inc_data_pat3 = torch.from_numpy(rawdata_inc3).to(device)
    mask1 = torch.from_numpy(missing_col1).to(device)
    mask2 = torch.from_numpy(missing_col2).to(device)
    mask3 = torch.from_numpy(np.logical_and(missing_col1,missing_col2)).to(device)
    for i in range(1):
      impu_noise = torch.empty(sum(np.logical_and(missing_row1,np.logical_not(missing_row2))), p_f+1, device=device)
      imputed_data = imputer1(inc_data_pat1.float(), mask1.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc1[:,missing_col1] = imputed_data.cpu().numpy()
      rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2))] = rawdata_inc1.copy()
      
      impu_noise = torch.empty(sum(np.logical_and(missing_row2,np.logical_not(missing_row1))), p_f+1, device=device)
      imputed_data = imputer2(inc_data_pat2.float(), mask2.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc2[:,missing_col2] = imputed_data.cpu().numpy()
      rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1))] = rawdata_inc2.copy()

      impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2)), p_f+1, device=device)
      imputed_data = imputer3(inc_data_pat3.float(), mask3.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data.cpu().numpy()
      rawdata[np.logical_and(missing_row1, missing_row2)] = rawdata_inc3.copy()
  
  alist = []
  # ilr=0.0002
  # clr=0.0002
  for j in range(1+MI):
    rawdata_pat1 = rawdata[np.logical_and(missing_row1,missing_row2==False),:].copy()
    train_pat1 = rawdata[np.logical_or(missing_row1==False,missing_row2),:].copy()
    train_pat1 = Generate_data_model_2(train_pat1, missing_col1)
    train_pat1_loader1 = DataLoader(train_pat1, batch_size=Batch_size, shuffle=True)
    train_pat1_loader2 = DataLoader(train_pat1, batch_size=Batch_size, shuffle=True)
    
    imputer1 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col1)).to(device) 
    critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)    
    imputer_optimizer = optim.Adam(imputer1.parameters(), lr=ilr, betas=(.5, .9))
    critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
    scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
    scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
    
    critic_updates = 0
    for epoch in range(1,1+2*epochs):
      for cc, mask in train_pat1_loader1:      
        cc = cc.to(device).float()                           
        batch_size = cc.shape[0]
        impu_noise = torch.empty(batch_size, p_f+1, device=device)
      
        cc_1, mask1 = next(iter(train_pat1_loader2))
        cc_1 = cc_1[0:batch_size].to(device).float()
        mask1 = mask1[0:batch_size].to(device).float()
        impu_noise.normal_(mean=0,std=1)
        imputed_data1 = imputer1(cc_1, mask1, impu_noise)
        cc_hat1 = cc_1.clone()
        cc_hat1[:,missing_col1] = imputed_data1
        update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
        update_critic(cc, cc_hat1)
        critic_updates += 1

        if critic_updates == n_critic:
          critic_updates = 0
          reconstruction_loss = 0

          impu_noise.normal_(mean=0,std=1)
          imputed_data1 = imputer1(cc_1, mask1, impu_noise)
          cc_hat1 = cc_1.clone()
          cc_hat1[:,missing_col1] = imputed_data1
          reconstruction_loss += l1_loss(imputed_data1, cc_1[:,missing_col1].float())
          loss = -critic(cc_hat1).mean() + weight*reconstruction_loss
          imputer1.zero_grad()
          loss.backward()
          imputer_optimizer.step()
      scheduler1.step()
      scheduler2.step()
    
    with torch.no_grad():
      imputer1.eval()
      inc_data_pat1 = torch.from_numpy(rawdata_pat1).to(device)
      mask1 = torch.from_numpy(missing_col1).to(device)
      for i in range(1):
        torch.manual_seed(j)
        impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2==False)), p_f+1, device=device)
        imputed_data = imputer1(inc_data_pat1.float(), mask1.float(), impu_noise.normal_(mean=0,std=1)) 
        rawdata_pat1[:,missing_col1] = imputed_data.cpu().numpy()
        rawdata[np.logical_and(missing_row1,missing_row2==False)] = rawdata_pat1.copy()

    rawdata_pat2 = rawdata[np.logical_and(missing_row1==False,missing_row2),:].copy()
    train_pat2 = rawdata[np.logical_or(missing_row1,missing_row2==False),:].copy()
    train_pat2 =  Generate_data_model_2(train_pat2, missing_col2)
    
    train_pat2_loader1 = DataLoader(train_pat2, batch_size=Batch_size, shuffle=True)
    train_pat2_loader2 = DataLoader(train_pat2, batch_size=Batch_size, shuffle=True)
    imputer2 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(missing_col2)).to(device) 
    critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
    imputer_optimizer = optim.Adam(imputer2.parameters(), lr=ilr, betas=(.5, .9))
    critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
    scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
    scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
    critic_updates = 0

    for epoch in range(1,1+2*epochs):
      for cc, mask in train_pat2_loader1:      
        cc = cc.to(device).float()                           
        batch_size = cc.shape[0]
        impu_noise = torch.empty(batch_size, p_f+1, device=device)
      
        cc_2, mask2 = next(iter(train_pat2_loader2))
        cc_2 = cc_2[0:batch_size].to(device).float()
        mask2 = mask2[0:batch_size].to(device).float()
        impu_noise.normal_(mean=0,std=1)
        imputed_data2 = imputer2(cc_2, mask2, impu_noise)
        cc_hat2 = cc_2.clone()
        cc_hat2[:,missing_col2] = imputed_data2

        update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
        update_critic(cc, cc_hat2)
        critic_updates += 1

        if critic_updates == n_critic:
          critic_updates = 0
          reconstruction_loss = 0
          
          impu_noise.normal_(mean=0,std=1)
          imputed_data2 = imputer2(cc_2, mask2, impu_noise)
          cc_hat2 = cc_2.clone()
          cc_hat2[:,missing_col2] = imputed_data2
          reconstruction_loss += l1_loss(imputed_data2, cc_2[:,missing_col2].float())
          loss = -critic(cc_hat2).mean() + weight*reconstruction_loss
          imputer2.zero_grad()
          loss.backward()
          imputer_optimizer.step()
      scheduler1.step()
      scheduler2.step()

    with torch.no_grad():
      imputer2.eval()
      inc_data_pat2 = torch.from_numpy(rawdata_pat2).to(device)
      mask2 = torch.from_numpy(missing_col2).to(device)
      for i in range(1):
        torch.manual_seed(j)
        impu_noise = torch.empty(sum(np.logical_and(missing_row1==False,missing_row2)), p_f+1, device=device)
        imputed_data = imputer2(inc_data_pat2.float(), mask2.float(), impu_noise.normal_(mean=0,std=1)) 
        rawdata_pat2[:,missing_col2] = imputed_data.cpu().numpy()
        rawdata[np.logical_and(missing_row1==False,missing_row2)] = rawdata_pat2.copy()
    
    rawdata_pat3 = rawdata[np.logical_and(missing_row1,missing_row2),:].copy()
    train_pat3 = rawdata[np.logical_or(missing_row1==False,missing_row2==False),:].copy()
    train_pat3 =  Generate_data_model_2(train_pat3, np.logical_or(missing_col1,missing_col2))
    
    train_pat3_loader1 = DataLoader(train_pat3, batch_size=Batch_size, shuffle=True)
    train_pat3_loader2 = DataLoader(train_pat3, batch_size=Batch_size, shuffle=True)
    imputer3 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=sum(np.logical_or(missing_col1,missing_col2))).to(device) 
    critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
    imputer_optimizer = optim.Adam(imputer3.parameters(), lr=ilr, betas=(.5, .9))
    critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
    scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
    scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
    critic_updates = 0

    for epoch in range(1,1+2*epochs):
      for cc, mask in train_pat3_loader1:      
        cc = cc.to(device).float()                           
        batch_size = cc.shape[0]
        impu_noise = torch.empty(batch_size, p_f+1, device=device)
      
        cc_3, mask3 = next(iter(train_pat3_loader2))
        cc_3 = cc_3[0:batch_size].to(device).float()
        mask3 = mask3[0:batch_size].to(device).float()
        impu_noise.normal_(mean=0,std=1)
        imputed_data3 = imputer3(cc_3, mask3, impu_noise)
        cc_hat3 = cc_3.clone()
        cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3

        update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
        update_critic(cc, cc_hat3)
        critic_updates += 1

        if critic_updates == n_critic:
          critic_updates = 0
          reconstruction_loss = 0
          
          impu_noise.normal_(mean=0,std=1)
          imputed_data3 = imputer3(cc_3, mask3, impu_noise)
          cc_hat3 = cc_3.clone()
          cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3
          reconstruction_loss += l1_loss(imputed_data3, cc_3[:,np.logical_or(missing_col1,missing_col2)].float())
          loss = -critic(cc_hat3).mean() + weight*reconstruction_loss
          imputer3.zero_grad()
          loss.backward()
          imputer_optimizer.step()
      scheduler1.step()
      scheduler2.step()

    with torch.no_grad():
      imputer3.eval()
      inc_data_pat3 = torch.from_numpy(rawdata_pat3).to(device)
      mask3 = torch.from_numpy(np.logical_or(missing_col1,missing_col2)).to(device)
      for i in range(1):
        torch.manual_seed(j)
        impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2)), p_f+1, device=device)
        imputed_data = imputer3(inc_data_pat3.float(), mask3.float(), impu_noise.normal_(mean=0,std=1)) 
        rawdata_pat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data.cpu().numpy()
        rawdata[np.logical_and(missing_row1,missing_row2)] = rawdata_pat3.copy()

    if j > 0:
      alist.append(rawdata.copy())
  return alist

## test

In [ ]:
Model_2_SD=[]
Model_2_SE_list=[]
Model_2_SE=0
Model_2_MSE=0
Model_2_estimate=0
Model_2_time=0

MI=3
batch_size=1024
ilr=1e-3 
clr=1e-3 
gamma=0.9
weight=0.1 
gp_lambda=10 
epochs=300

a0=-1
a1=-3
a2=3
a3=-1
a4=-3
a5=2
Monte_Carlo = 100


for iteration in range(1,1+Monte_Carlo):
  sample,missing_row1,missing_row2,missing_col1,missing_col2,y=data_G_linear(n=n,p=p,data=adni_data,seed=iteration,a0=a0,a1=a1,a2=a2,a3=a3,a4=a4,a5=a5)
  # print(sum(missing_row))
  truth1=sample[missing_row1==True,:][:,missing_col1==True].copy()
  truth2=sample[missing_row2==True,:][:,missing_col2==True].copy()
  
  mis1 = sample[:,missing_col1==True].copy()
  mis2 = sample[:,missing_col2==True].copy()
  mis1[missing_row1==True,:] = 0
  mis2[missing_row2==True,:] = 0
  sample[:,missing_col1==True]=mis1
  sample[:,missing_col2==True]=mis2
  rawdata = sample.copy()

  s=time.time()
  # alist=[]
  # for i in range(MI):
  #   data = mlp_MIGAN2(rawdata.copy(),missing_row1,missing_row2,missing_col1,missing_col2,n=n,p_f=p,Batch_size=batch_size,ilr=ilr,clr=clr,gamma=gamma,weight=weight,gp_lambda=gp_lambda,epochs=epochs,MI_seed=MI)[0]
  #   alist.append(data)
  
  alist = mlp_MIGAN2_iterative2(rawdata.copy(),missing_row1,missing_row2,missing_col1,missing_col2,n=n,p_f=p,Batch_size=batch_size,ilr=ilr,clr=clr,gamma=gamma,weight=weight,gp_lambda=gp_lambda,epochs=epochs,MI=MI)
  
  mse,b,c,se=data_to_stats(alist,truth1,truth2,missing_row1,missing_row2,missing_col1,missing_col2)

  print('Model_2 cost ',time.time()-s,' sec \n','MSE:', mse,'; Estimate: ',b[1])
  Model_2_SE_list.append(se)
  Model_2_SE=(Model_2_SE*(iteration-1)+se)/iteration
  Model_2_SD.append(b[1])
  Model_2_MSE=(Model_2_MSE*(iteration-1)+mse)/iteration
  Model_2_estimate=(Model_2_estimate*(iteration-1)+b[1])/iteration
  Model_2_time = (Model_2_time*(iteration-1)+time.time()-s)/iteration

  if iteration%5==0:
    print('iteration: ',iteration, 'Model_2_MSE: ',Model_2_MSE,'Model_2_estimate: ',Model_2_estimate,'Model_2_SE: ',Model_2_SE)
    
    Model_2_StD=np.std(Model_2_SD, dtype=np.float64)
    print('Model_2_StD: ',Model_2_StD)
    # Model_2_SE_median=statistics.median(Model_2_SE_list)
    # print('Model_2_SE_median: ',Model_2_SE_median)
    print('Model_2 average cost:', Model_2_time/MI)

    print('n:', n, ' p: ',p, ' Monte Carlo:',Monte_Carlo, 'MI:',MI,'batch_size:',batch_size,'ilr:',ilr,'clr:',clr,'gamma:',gamma,'weight:',weight,'gp_lambda:',gp_lambda,'epochs:',epochs)
    print('a0=',a0,' a1=', a1,' a2=',a2,' a3=',a3,' a4=',a4,' a5=',a5, 'q=',q)

# MIGAN1

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import grad
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import WeightedRandomSampler
from torchvision import datasets, transforms
import torch.nn.functional as F

In [ ]:
class Generate_data_model(Dataset):   
  def __init__(self, array, transform=None):  
    self.array = array
    self.transform = transform

  def __len__(self):
    return len(self.array)

  def __getitem__(self, index):
    return self.array[index]

class Generate_data_model_2(Dataset):   
  def __init__(self, array, mask, transform=None):  
    self.array = array
    self.mask = mask             
    self.transform = transform

  def __len__(self):
    return len(self.array)

  def __getitem__(self, index):
    return self.array[index], self.mask

class Imputer_model_2(nn.Module):
  def __init__(self, n1=251, n2=200, n3=100, n4=100):
    super().__init__()
    self.fc = nn.Sequential(
        nn.Linear(n1, n2),
        # nn.ReLU(),
        nn.Tanh(),
        nn.Linear(n2, n3),
        # nn.ReLU(),
        nn.Tanh(),
        nn.Linear(n3, n4),
    )
  def forward(self, data, mask, noise):
    net = data * (1 - mask) + noise * mask
    net = net.view(data.shape[0], -1)
    net = self.fc(net.float())
    net = net.view(data.shape[0], -1)
    return net

class Critic_model_2(nn.Module):
  def __init__(self, n1=251, n2=200, n3=50, n4=1):  #n1=251, n2=100, n3=20, n4=1
    super().__init__()
    self.fc = nn.Sequential(
        nn.Linear(n1, n2),
        nn.ReLU(),   
        # nn.Tanh(),
        nn.Linear(n2, n3),
        nn.ReLU(),   
        # nn.Tanh(),
        nn.Linear(n3, n4),
    )
  def forward(self, x):
    out = self.fc(x)
    return out

class CriticUpdater_model:
  def __init__(self, critic, critic_optimizer, batch_size=8, gp_lambda=10, device='cuda:0'):
    self.critic = critic
    self.critic_optimizer = critic_optimizer
    self.gp_lambda = gp_lambda
    # Interpolation coefficient
    self.eps = torch.empty(batch_size, 1,device=device)
    # For computing the gradient penalty
    self.ones = torch.ones(batch_size, 1).to(device)

  def __call__(self, real, fake, device='cuda:0'):
    real = real.detach()
    fake = fake.detach()
    self.critic.zero_grad()
    self.eps.uniform_(0, 1)
    interp = (self.eps * real + (1 - self.eps) * fake).to(device).requires_grad_()
    grad_d = grad(outputs=self.critic(interp.float()), inputs=interp, grad_outputs=self.ones,create_graph=True)[0]  #grad_outputs=self.ones,create_graph=True
    grad_d = grad_d.view(real.shape[0], -1)
    grad_penalty = ((grad_d.norm(dim=1) - 1)**2).mean() * self.gp_lambda
    w_dist = self.critic(fake.float()).mean() - self.critic(real.float()).mean()
    loss = w_dist + grad_penalty
    loss.backward()
    self.critic_optimizer.step()

class CriticUpdater_model_2:
  def __init__(self, critic, critic_optimizer, batch_size=8, gp_lambda=10, device='cuda:0'):
    self.critic = critic
    self.critic_optimizer = critic_optimizer
    self.gp_lambda = gp_lambda
    # Interpolation coefficient
    self.eps = torch.empty(batch_size, 1,device=device)
    # For computing the gradient penalty
    self.ones = torch.ones(batch_size, 1).to(device)

  def __call__(self, real, fake1, fake2, fake3, device='cuda:0'):
    real = real.detach()
    fake1 = fake1.detach()
    fake2 = fake2.detach()
    fake3 = fake3.detach()
    self.critic.zero_grad()
    self.eps.uniform_(0, 1)
    interp1 = (self.eps * real + (1 - self.eps) * fake1).to(device).requires_grad_()
    self.eps.uniform_(0, 1)
    interp2 = (self.eps * real + (1 - self.eps) * fake2).to(device).requires_grad_()
    self.eps.uniform_(0, 1)
    interp3 = (self.eps * real + (1 - self.eps) * fake3).to(device).requires_grad_()
    grad_d1 = grad(outputs=self.critic(interp1.float()), inputs=interp1, grad_outputs=self.ones,create_graph=True)[0].view(real.shape[0], -1)  #grad_outputs=self.ones,create_graph=True
    grad_d2 = grad(outputs=self.critic(interp2.float()), inputs=interp2, grad_outputs=self.ones,create_graph=True)[0].view(real.shape[0], -1)
    grad_d3 = grad(outputs=self.critic(interp3.float()), inputs=interp3, grad_outputs=self.ones,create_graph=True)[0].view(real.shape[0], -1)
    grad_penalty1 = ((grad_d1.norm(dim=1) - 1)**2).mean() * self.gp_lambda
    grad_penalty2 = ((grad_d2.norm(dim=1) - 1)**2).mean() * self.gp_lambda
    grad_penalty3 = ((grad_d3.norm(dim=1) - 1)**2).mean() * self.gp_lambda
    w_dist1 = self.critic(fake1.float()).mean() - self.critic(real.float()).mean()
    w_dist2 = self.critic(fake2.float()).mean() - self.critic(real.float()).mean()
    w_dist3 = self.critic(fake3.float()).mean() - self.critic(real.float()).mean()
    loss = w_dist1 + w_dist2 + w_dist3 + grad_penalty1 + grad_penalty2 + grad_penalty3 
    loss.backward()
    self.critic_optimizer.step()

def mlp_MIGAN1(missing_data, missing_row1,missing_row2,missing_col1,missing_col2, n=200, p_f=250, device='cuda:0', Batch_size=16, ilr=1e-3, clr=1e-3, gamma=0.2, weight=0.1, gp_lambda=10, epochs=300, MI_seed=1):
  rawdata = missing_data.copy()
  rawdata_inc1 = rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2)), :].copy()
  rawdata_inc2 = rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1)), :].copy()
  rawdata_inc3 = rawdata[np.logical_and(missing_row1,missing_row2), :].copy()
  cc = torch.from_numpy(rawdata.copy()[np.logical_or(missing_row1,missing_row2)==0, :])

  cc_data = Generate_data_model(cc)
  cc_data_pat1 =  Generate_data_model_2(cc, missing_col1)
  cc_data_pat2 =  Generate_data_model_2(cc, missing_col2)
  cc_data_pat3 =  Generate_data_model_2(cc, np.logical_or(missing_col1,missing_col2))

  cc_data_loader = DataLoader(cc_data, batch_size=Batch_size, shuffle=True)
  cc_data_loader1 = DataLoader(cc_data_pat1, batch_size=Batch_size, shuffle=True)
  cc_data_loader2 = DataLoader(cc_data_pat2, batch_size=Batch_size, shuffle=True)
  cc_data_loader3 = DataLoader(cc_data_pat3, batch_size=Batch_size, shuffle=True)
  
  imputer1 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device)
  imputer2 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device)
  imputer3 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device)  

  critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)    
  imputer_params = list(imputer1.parameters()) + list(imputer2.parameters()) + list(imputer3.parameters())
  imputer_optimizer = optim.Adam(imputer_params, lr=ilr, betas=(.5, .9))
  critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
  scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
  scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
  mse_loss = torch.nn.MSELoss(reduction='mean')
  l1_loss = nn.L1Loss(reduction='mean')

  critic_updates = 0
  n_critic = 5

  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
     
      cc_1, mask1 = next(iter(cc_data_loader1))
      cc_1 = cc_1[0:batch_size].to(device).float()
      mask1 = mask1[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data1 = imputer1(cc_1, mask1, impu_noise)
      cc_hat1 = cc_1.clone()
      cc_hat1[:,missing_col1] = imputed_data1[:,missing_col1]

      cc_2, mask2 = next(iter(cc_data_loader2))
      cc_2 = cc_2[0:batch_size].to(device).float()
      mask2 = mask2[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data2 = imputer2(cc_2, mask2, impu_noise)
      cc_hat2 = cc_2.clone()
      cc_hat2[:,missing_col2] = imputed_data2[:,missing_col2]

      cc_3, mask3 = next(iter(cc_data_loader3))
      cc_3 = cc_3[0:batch_size].to(device).float()
      mask3 = mask3[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data3 = imputer3(cc_3, mask3, impu_noise)
      cc_hat3 = cc_3.clone()
      cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3[:,np.logical_or(missing_col1,missing_col2)]
      
      update_critic = CriticUpdater_model_2(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat1, cc_hat2, cc_hat3)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0

        impu_noise.normal_(mean=0,std=1)
        imputed_data1 = imputer1(cc_1, mask1, impu_noise)
        cc_hat1 = cc_1.clone()
        cc_hat1[:,missing_col1] = imputed_data1[:,missing_col1]
        reconstruction_loss += l1_loss(imputed_data1, cc_1)
        
        impu_noise.normal_(mean=0,std=1)
        imputed_data2 = imputer2(cc_2, mask2, impu_noise)
        cc_hat2 = cc_2.clone()
        cc_hat2[:,missing_col2] = imputed_data2[:,missing_col2]
        reconstruction_loss += l1_loss(imputed_data2, cc_2)

        impu_noise.normal_(mean=0,std=1)
        imputed_data3 = imputer3(cc_3, mask3, impu_noise)
        cc_hat3 = cc_3.clone()
        cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3[:,np.logical_or(missing_col1,missing_col2)]
        reconstruction_loss += l1_loss(imputed_data3, cc_3)

        loss = -critic(cc_hat1).mean()-critic(cc_hat2).mean()-critic(cc_hat3).mean() + weight*reconstruction_loss
        
        imputer1.zero_grad()
        imputer2.zero_grad()
        imputer3.zero_grad()
        loss.backward()
        imputer_optimizer.step()

    scheduler1.step()
    scheduler2.step()

  alist = []
  with torch.no_grad():
    imputer1.eval()
    imputer2.eval()
    imputer3.eval()
    inc_data_pat1 = torch.from_numpy(rawdata_inc1).to(device)
    inc_data_pat2 = torch.from_numpy(rawdata_inc2).to(device)
    inc_data_pat3 = torch.from_numpy(rawdata_inc3).to(device)
    mask1 = torch.from_numpy(missing_col1).to(device)
    mask2 = torch.from_numpy(missing_col2).to(device)
    mask3 = torch.from_numpy(np.logical_and(missing_col1,missing_col2)).to(device)
    for i in range(1):
      torch.manual_seed(MI_seed)
      impu_noise = torch.empty(sum(np.logical_and(missing_row1,np.logical_not(missing_row2))), p_f+1, device=device)
      imputed_data = imputer1(inc_data_pat1.float(), mask1.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc1[:,missing_col1] = imputed_data.cpu().numpy()[:,missing_col1]
      # print(sum(np.isnan(imputed_data.cpu().numpy())))
      rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2))] = rawdata_inc1.copy()
      
      impu_noise = torch.empty(sum(np.logical_and(missing_row2,np.logical_not(missing_row1))), p_f+1, device=device)
      imputed_data = imputer2(inc_data_pat2.float(), mask2.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc2[:,missing_col2] = imputed_data.cpu().numpy()[:,missing_col2]
      # print(sum(np.isnan(imputed_data.cpu().numpy())))
      rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1))] = rawdata_inc2.copy()

      impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2)), p_f+1, device=device)
      imputed_data = imputer3(inc_data_pat3.float(), mask3.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data.cpu().numpy()[:,np.logical_or(missing_col1,missing_col2)]
      # print(sum(np.isnan(imputed_data.cpu().numpy())))
      rawdata[np.logical_and(missing_row1, missing_row2)] = rawdata_inc3.copy()

      alist.append(rawdata.copy())
  return alist

In [ ]:
def mlp_MIGAN1_version2(missing_data, missing_row1,missing_row2,missing_col1,missing_col2, n=200, p_f=250, device='cuda:0', Batch_size=16, ilr=1e-3, clr=1e-3, gamma=0.2, weight=0.1, gp_lambda=10, epochs=300, MI_seed=10):
  rawdata = missing_data.copy()
  rawdata_inc1 = rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2)), :].copy()
  rawdata_inc2 = rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1)), :].copy()
  rawdata_inc3 = rawdata[np.logical_and(missing_row1,missing_row2), :].copy()
  cc = torch.from_numpy(rawdata.copy()[np.logical_or(missing_row1,missing_row2)==0, :])

  cc_data = Generate_data_model(cc)
  cc_data_pat1 =  Generate_data_model_2(cc, missing_col1)
  cc_data_pat2 =  Generate_data_model_2(cc, missing_col2)
  cc_data_pat3 =  Generate_data_model_2(cc, np.logical_or(missing_col1,missing_col2))

  cc_data_loader = DataLoader(cc_data, batch_size=Batch_size, shuffle=True)
  cc_data_loader1 = DataLoader(cc_data_pat1, batch_size=Batch_size, shuffle=True)
  cc_data_loader2 = DataLoader(cc_data_pat2, batch_size=Batch_size, shuffle=True)
  cc_data_loader3 = DataLoader(cc_data_pat3, batch_size=Batch_size, shuffle=True)
  
  mse_loss = torch.nn.MSELoss(reduction='mean')
  l1_loss = nn.L1Loss(reduction='mean')
  critic_updates = 0
  n_critic = 5
  
  imputer1 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device) 
  critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)    
  imputer_optimizer = optim.Adam(imputer1.parameters(), lr=ilr, betas=(.5, .9))
  critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
  scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
  scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
  
  critic_updates = 0
  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
    
      cc_1, mask1 = next(iter(cc_data_loader1))
      cc_1 = cc_1[0:batch_size].to(device).float()
      mask1 = mask1[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data1 = imputer1(cc_1, mask1, impu_noise)
      cc_hat1 = cc_1.clone()
      cc_hat1[:,missing_col1] = imputed_data1[:,missing_col1]
      update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat1)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0

        impu_noise.normal_(mean=0,std=1)
        imputed_data1 = imputer1(cc_1, mask1, impu_noise)
        cc_hat1 = cc_1.clone()
        cc_hat1[:,missing_col1] = imputed_data1[:,missing_col1]
        reconstruction_loss += l1_loss(imputed_data1, cc_1.float())
        loss = -critic(cc_hat1).mean() + weight*reconstruction_loss
        imputer1.zero_grad()
        loss.backward()
        imputer_optimizer.step()
    scheduler1.step()
    scheduler2.step()
  
  imputer2 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device) 
  critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
  imputer_optimizer = optim.Adam(imputer2.parameters(), lr=ilr, betas=(.5, .9))
  critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
  scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
  scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
  critic_updates = 0

  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
    
      cc_2, mask2 = next(iter(cc_data_loader2))
      cc_2 = cc_2[0:batch_size].to(device).float()
      mask2 = mask2[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data2 = imputer2(cc_2, mask2, impu_noise)
      cc_hat2 = cc_2.clone()
      cc_hat2[:,missing_col2] = imputed_data2[:,missing_col2]

      update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat2)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0
        
        impu_noise.normal_(mean=0,std=1)
        imputed_data2 = imputer2(cc_2, mask2, impu_noise)
        cc_hat2 = cc_2.clone()
        cc_hat2[:,missing_col2] = imputed_data2[:,missing_col2]
        reconstruction_loss += l1_loss(imputed_data2, cc_2.float())
        loss = -critic(cc_hat2).mean() + weight*reconstruction_loss
        imputer2.zero_grad()
        loss.backward()
        imputer_optimizer.step()
    scheduler1.step()
    scheduler2.step()


  imputer3 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device) 
  critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
  imputer_optimizer = optim.Adam(imputer3.parameters(), lr=ilr, betas=(.5, .9))
  critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
  scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
  scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
  critic_updates = 0

  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
    
      cc_3, mask3 = next(iter(cc_data_loader3))
      cc_3 = cc_3[0:batch_size].to(device).float()
      mask3 = mask3[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data3 = imputer3(cc_3, mask3, impu_noise)
      cc_hat3 = cc_3.clone()
      cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3[:,np.logical_or(missing_col1,missing_col2)]

      update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat3)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0
        
        impu_noise.normal_(mean=0,std=1)
        imputed_data3 = imputer3(cc_3, mask3, impu_noise)
        cc_hat3 = cc_3.clone()
        cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3[:,np.logical_or(missing_col1,missing_col2)]
        reconstruction_loss += l1_loss(imputed_data3, cc_3.float())
        loss = -critic(cc_hat3).mean() + weight*reconstruction_loss
        imputer3.zero_grad()
        loss.backward()
        imputer_optimizer.step()
    scheduler1.step()
    scheduler2.step()
  
  alist=[]
  with torch.no_grad():
    imputer1.eval()
    imputer2.eval()
    imputer3.eval()
    inc_data_pat1 = torch.from_numpy(rawdata_inc1).to(device)
    inc_data_pat2 = torch.from_numpy(rawdata_inc2).to(device)
    inc_data_pat3 = torch.from_numpy(rawdata_inc3).to(device)
    mask1 = torch.from_numpy(missing_col1).to(device)
    mask2 = torch.from_numpy(missing_col2).to(device)
    mask3 = torch.from_numpy(np.logical_and(missing_col1,missing_col2)).to(device)
    for i in range(1):
      torch.manual_seed(MI_seed)
      impu_noise = torch.empty(sum(np.logical_and(missing_row1,np.logical_not(missing_row2))), p_f+1, device=device)
      imputed_data = imputer1(inc_data_pat1.float(), mask1.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc1[:,missing_col1] = imputed_data.cpu().numpy()[:,missing_col1]
      rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2))] = rawdata_inc1.copy()
      
      impu_noise = torch.empty(sum(np.logical_and(missing_row2,np.logical_not(missing_row1))), p_f+1, device=device)
      imputed_data = imputer2(inc_data_pat2.float(), mask2.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc2[:,missing_col2] = imputed_data.cpu().numpy()[:,missing_col2]
      rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1))] = rawdata_inc2.copy()

      impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2)), p_f+1, device=device)
      imputed_data = imputer3(inc_data_pat3.float(), mask3.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data.cpu().numpy()[:,np.logical_or(missing_col1,missing_col2)]
      rawdata[np.logical_and(missing_row1, missing_row2)] = rawdata_inc3.copy()

      alist.append(rawdata.copy())
  return alist

## iterative approach, row pattern, 1

In [ ]:
def mlp_MIGAN1_iterative2(missing_data, missing_row1,missing_row2,missing_col1,missing_col2, n=200, p_f=250, device='cuda:0', Batch_size=16, ilr=1e-3, clr=1e-3, gamma=0.2, weight=0.1, gp_lambda=10, epochs=300, MI=10):
  rawdata = missing_data.copy()
  rawdata_inc1 = rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2)), :].copy()
  rawdata_inc2 = rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1)), :].copy()
  rawdata_inc3 = rawdata[np.logical_and(missing_row1,missing_row2), :].copy()
  cc = torch.from_numpy(rawdata.copy()[np.logical_or(missing_row1,missing_row2)==0, :])

  cc_data = Generate_data_model(cc)
  cc_data_pat1 =  Generate_data_model_2(cc, missing_col1)
  cc_data_pat2 =  Generate_data_model_2(cc, missing_col2)
  cc_data_pat3 =  Generate_data_model_2(cc, np.logical_or(missing_col1,missing_col2))

  cc_data_loader = DataLoader(cc_data, batch_size=Batch_size, shuffle=True)
  cc_data_loader1 = DataLoader(cc_data_pat1, batch_size=Batch_size, shuffle=True)
  cc_data_loader2 = DataLoader(cc_data_pat2, batch_size=Batch_size, shuffle=True)
  cc_data_loader3 = DataLoader(cc_data_pat3, batch_size=Batch_size, shuffle=True)
  
  mse_loss = torch.nn.MSELoss(reduction='mean')
  l1_loss = nn.L1Loss(reduction='mean')
  critic_updates = 0
  n_critic = 5
  
  imputer1 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device) 
  critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)    
  imputer_optimizer = optim.Adam(imputer1.parameters(), lr=ilr, betas=(.5, .9))
  critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
  scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
  scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
  
  critic_updates = 0
  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
    
      cc_1, mask1 = next(iter(cc_data_loader1))
      cc_1 = cc_1[0:batch_size].to(device).float()
      mask1 = mask1[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data1 = imputer1(cc_1, mask1, impu_noise)
      cc_hat1 = cc_1.clone()
      cc_hat1[:,missing_col1] = imputed_data1[:,missing_col1]
      update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat1)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0

        impu_noise.normal_(mean=0,std=1)
        imputed_data1 = imputer1(cc_1, mask1, impu_noise)
        cc_hat1 = cc_1.clone()
        cc_hat1[:,missing_col1] = imputed_data1[:,missing_col1]
        reconstruction_loss += l1_loss(imputed_data1, cc_1.float())
        loss = -critic(cc_hat1).mean() + weight*reconstruction_loss
        imputer1.zero_grad()
        loss.backward()
        imputer_optimizer.step()
    scheduler1.step()
    scheduler2.step()
  
  imputer2 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device) 
  critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
  imputer_optimizer = optim.Adam(imputer2.parameters(), lr=ilr, betas=(.5, .9))
  critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
  scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
  scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
  critic_updates = 0

  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
    
      cc_2, mask2 = next(iter(cc_data_loader2))
      cc_2 = cc_2[0:batch_size].to(device).float()
      mask2 = mask2[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data2 = imputer2(cc_2, mask2, impu_noise)
      cc_hat2 = cc_2.clone()
      cc_hat2[:,missing_col2] = imputed_data2[:,missing_col2]

      update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat2)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0
        
        impu_noise.normal_(mean=0,std=1)
        imputed_data2 = imputer2(cc_2, mask2, impu_noise)
        cc_hat2 = cc_2.clone()
        cc_hat2[:,missing_col2] = imputed_data2[:,missing_col2]
        reconstruction_loss += l1_loss(imputed_data2, cc_2.float())
        loss = -critic(cc_hat2).mean() + weight*reconstruction_loss
        imputer2.zero_grad()
        loss.backward()
        imputer_optimizer.step()
    scheduler1.step()
    scheduler2.step()

  imputer3 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device) 
  critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
  imputer_optimizer = optim.Adam(imputer3.parameters(), lr=ilr, betas=(.5, .9))
  critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
  scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
  scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
  critic_updates = 0

  for epoch in range(1,1+epochs):
    for cc in cc_data_loader:      
      cc = cc.to(device).float()                           
      batch_size = cc.shape[0]
      impu_noise = torch.empty(batch_size, p_f+1, device=device)
    
      cc_3, mask3 = next(iter(cc_data_loader3))
      cc_3 = cc_3[0:batch_size].to(device).float()
      mask3 = mask3[0:batch_size].to(device).float()
      impu_noise.normal_(mean=0,std=1)
      imputed_data3 = imputer3(cc_3, mask3, impu_noise)
      cc_hat3 = cc_3.clone()
      cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3[:,np.logical_or(missing_col1,missing_col2)]

      update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
      update_critic(cc, cc_hat3)
      critic_updates += 1

      if critic_updates == n_critic:
        critic_updates = 0
        reconstruction_loss = 0
        
        impu_noise.normal_(mean=0,std=1)
        imputed_data3 = imputer3(cc_3, mask3, impu_noise)
        cc_hat3 = cc_3.clone()
        cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3[:,np.logical_or(missing_col1,missing_col2)]
        reconstruction_loss += l1_loss(imputed_data3, cc_3.float())
        loss = -critic(cc_hat3).mean() + weight*reconstruction_loss
        imputer3.zero_grad()
        loss.backward()
        imputer_optimizer.step()
    scheduler1.step()
    scheduler2.step()
  
  with torch.no_grad():
    imputer1.eval()
    imputer2.eval()
    imputer3.eval()
    inc_data_pat1 = torch.from_numpy(rawdata_inc1).to(device)
    inc_data_pat2 = torch.from_numpy(rawdata_inc2).to(device)
    inc_data_pat3 = torch.from_numpy(rawdata_inc3).to(device)
    mask1 = torch.from_numpy(missing_col1).to(device)
    mask2 = torch.from_numpy(missing_col2).to(device)
    mask3 = torch.from_numpy(np.logical_and(missing_col1,missing_col2)).to(device)
    for i in range(1):
      impu_noise = torch.empty(sum(np.logical_and(missing_row1,np.logical_not(missing_row2))), p_f+1, device=device)
      imputed_data = imputer1(inc_data_pat1.float(), mask1.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc1[:,missing_col1] = imputed_data.cpu().numpy()[:,missing_col1]
      rawdata[np.logical_and(missing_row1,np.logical_not(missing_row2))] = rawdata_inc1.copy()
      
      impu_noise = torch.empty(sum(np.logical_and(missing_row2,np.logical_not(missing_row1))), p_f+1, device=device)
      imputed_data = imputer2(inc_data_pat2.float(), mask2.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc2[:,missing_col2] = imputed_data.cpu().numpy()[:,missing_col2]
      rawdata[np.logical_and(missing_row2,np.logical_not(missing_row1))] = rawdata_inc2.copy()

      impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2)), p_f+1, device=device)
      imputed_data = imputer3(inc_data_pat3.float(), mask3.float(), impu_noise.normal_(mean=0,std=1)) 
      rawdata_inc3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data.cpu().numpy()[:,np.logical_or(missing_col1,missing_col2)]
      rawdata[np.logical_and(missing_row1, missing_row2)] = rawdata_inc3.copy()

  
  alist = []
  for j in range(1+MI):
    rawdata_pat1 = rawdata[np.logical_and(missing_row1,missing_row2==False),:].copy()
    train_pat1 = rawdata[np.logical_or(missing_row1==False,missing_row2),:].copy()
    train_pat1 = Generate_data_model_2(train_pat1, missing_col1)
    train_pat1_loader1 = DataLoader(train_pat1, batch_size=Batch_size, shuffle=True)
    train_pat1_loader2 = DataLoader(train_pat1, batch_size=Batch_size, shuffle=True)
    
    imputer1 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device) 
    critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)    
    imputer_optimizer = optim.Adam(imputer1.parameters(), lr=ilr, betas=(.5, .9))
    critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
    scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
    scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
    
    critic_updates = 0
    for epoch in range(1,1+epochs):
      for cc, mask in train_pat1_loader1:      
        cc = cc.to(device).float()                           
        batch_size = cc.shape[0]
        impu_noise = torch.empty(batch_size, p_f+1, device=device)
      
        cc_1, mask1 = next(iter(train_pat1_loader2))
        cc_1 = cc_1[0:batch_size].to(device).float()
        mask1 = mask1[0:batch_size].to(device).float()
        impu_noise.normal_(mean=0,std=1)
        imputed_data1 = imputer1(cc_1, mask1, impu_noise)
        cc_hat1 = cc_1.clone()
        cc_hat1[:,missing_col1] = imputed_data1[:,missing_col1]
        update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
        update_critic(cc, cc_hat1)
        critic_updates += 1

        if critic_updates == n_critic:
          critic_updates = 0
          reconstruction_loss = 0

          impu_noise.normal_(mean=0,std=1)
          imputed_data1 = imputer1(cc_1, mask1, impu_noise)
          cc_hat1 = cc_1.clone()
          cc_hat1[:,missing_col1] = imputed_data1[:,missing_col1]
          reconstruction_loss += l1_loss(imputed_data1, cc_1.float())
          loss = -critic(cc_hat1).mean() + weight*reconstruction_loss
          imputer1.zero_grad()
          loss.backward()
          imputer_optimizer.step()
      scheduler1.step()
      scheduler2.step()
    
    with torch.no_grad():
      imputer1.eval()
      inc_data_pat1 = torch.from_numpy(rawdata_pat1).to(device)
      mask1 = torch.from_numpy(missing_col1).to(device)
      for i in range(1):
        torch.manual_seed(j)
        impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2==False)), p_f+1, device=device)
        imputed_data = imputer1(inc_data_pat1.float(), mask1.float(), impu_noise.normal_(mean=0,std=1)) 
        rawdata_pat1[:,missing_col1] = imputed_data.cpu().numpy()[:,missing_col1]
        rawdata[np.logical_and(missing_row1,missing_row2==False)] = rawdata_pat1.copy()

    rawdata_pat2 = rawdata[np.logical_and(missing_row1==False,missing_row2),:].copy()
    train_pat2 = rawdata[np.logical_or(missing_row1,missing_row2==False),:].copy()
    train_pat2 =  Generate_data_model_2(train_pat2, missing_col2)
    
    train_pat2_loader1 = DataLoader(train_pat2, batch_size=Batch_size, shuffle=True)
    train_pat2_loader2 = DataLoader(train_pat2, batch_size=Batch_size, shuffle=True)
    imputer2 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device) 
    critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
    imputer_optimizer = optim.Adam(imputer2.parameters(), lr=ilr, betas=(.5, .9))
    critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
    scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
    scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
    critic_updates = 0

    for epoch in range(1,1+epochs):
      for cc, mask in train_pat2_loader1:      
        cc = cc.to(device).float()                           
        batch_size = cc.shape[0]
        impu_noise = torch.empty(batch_size, p_f+1, device=device)
      
        cc_2, mask2 = next(iter(train_pat2_loader2))
        cc_2 = cc_2[0:batch_size].to(device).float()
        mask2 = mask2[0:batch_size].to(device).float()
        impu_noise.normal_(mean=0,std=1)
        imputed_data2 = imputer2(cc_2, mask2, impu_noise)
        cc_hat2 = cc_2.clone()
        cc_hat2[:,missing_col2] = imputed_data2[:,missing_col2]

        update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
        update_critic(cc, cc_hat2)
        critic_updates += 1

        if critic_updates == n_critic:
          critic_updates = 0
          reconstruction_loss = 0
          
          impu_noise.normal_(mean=0,std=1)
          imputed_data2 = imputer2(cc_2, mask2, impu_noise)
          cc_hat2 = cc_2.clone()
          cc_hat2[:,missing_col2] = imputed_data2[:,missing_col2]
          reconstruction_loss += l1_loss(imputed_data2, cc_2.float())
          loss = -critic(cc_hat2).mean() + weight*reconstruction_loss
          imputer2.zero_grad()
          loss.backward()
          imputer_optimizer.step()
      scheduler1.step()
      scheduler2.step()

    with torch.no_grad():
      imputer2.eval()
      inc_data_pat2 = torch.from_numpy(rawdata_pat2).to(device)
      mask2 = torch.from_numpy(missing_col2).to(device)
      for i in range(1):
        torch.manual_seed(j)
        impu_noise = torch.empty(sum(np.logical_and(missing_row1==False,missing_row2)), p_f+1, device=device)
        imputed_data = imputer2(inc_data_pat2.float(), mask2.float(), impu_noise.normal_(mean=0,std=1)) 
        rawdata_pat2[:,missing_col2] = imputed_data.cpu().numpy()[:,missing_col2]
        rawdata[np.logical_and(missing_row1==False,missing_row2)] = rawdata_pat2.copy()
    
    rawdata_pat3 = rawdata[np.logical_and(missing_row1,missing_row2),:].copy()
    train_pat3 = rawdata[np.logical_or(missing_row1==False,missing_row2==False),:].copy()
    train_pat3 =  Generate_data_model_2(train_pat3, np.logical_or(missing_col1,missing_col2))
    
    train_pat3_loader1 = DataLoader(train_pat3, batch_size=Batch_size, shuffle=True)
    train_pat3_loader2 = DataLoader(train_pat3, batch_size=Batch_size, shuffle=True)
    imputer3 = Imputer_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=p_f+1).to(device) 
    critic = Critic_model_2(n1=p_f+1, n2=p_f+1, n3=p_f+1, n4=1).to(device)
    imputer_optimizer = optim.Adam(imputer3.parameters(), lr=ilr, betas=(.5, .9))
    critic_optimizer = optim.Adam(critic.parameters(), lr=clr, betas=(.5, .9))
    scheduler1 = optim.lr_scheduler.StepLR(imputer_optimizer, step_size=10, gamma=gamma)
    scheduler2 = optim.lr_scheduler.StepLR(critic_optimizer, step_size=10, gamma=gamma)
    critic_updates = 0

    for epoch in range(1,1+epochs):
      for cc, mask in train_pat3_loader1:      
        cc = cc.to(device).float()                           
        batch_size = cc.shape[0]
        impu_noise = torch.empty(batch_size, p_f+1, device=device)
      
        cc_3, mask3 = next(iter(train_pat3_loader2))
        cc_3 = cc_3[0:batch_size].to(device).float()
        mask3 = mask3[0:batch_size].to(device).float()
        impu_noise.normal_(mean=0,std=1)
        imputed_data3 = imputer3(cc_3, mask3, impu_noise)
        cc_hat3 = cc_3.clone()
        cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3[:,np.logical_or(missing_col1,missing_col2)]

        update_critic = CriticUpdater_model(critic, critic_optimizer, batch_size, device=device, gp_lambda=gp_lambda)
        update_critic(cc, cc_hat3)
        critic_updates += 1

        if critic_updates == n_critic:
          critic_updates = 0
          reconstruction_loss = 0
          
          impu_noise.normal_(mean=0,std=1)
          imputed_data3 = imputer3(cc_3, mask3, impu_noise)
          cc_hat3 = cc_3.clone()
          cc_hat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data3[:,np.logical_or(missing_col1,missing_col2)]
          reconstruction_loss += l1_loss(imputed_data3, cc_3.float())
          loss = -critic(cc_hat3).mean() + weight*reconstruction_loss
          imputer3.zero_grad()
          loss.backward()
          imputer_optimizer.step()
      scheduler1.step()
      scheduler2.step()

    with torch.no_grad():
      imputer3.eval()
      inc_data_pat3 = torch.from_numpy(rawdata_pat3).to(device)
      mask3 = torch.from_numpy(np.logical_or(missing_col1,missing_col2)).to(device)
      for i in range(1):
        torch.manual_seed(j)
        impu_noise = torch.empty(sum(np.logical_and(missing_row1,missing_row2)), p_f+1, device=device)
        imputed_data = imputer3(inc_data_pat3.float(), mask3.float(), impu_noise.normal_(mean=0,std=1)) 
        rawdata_pat3[:,np.logical_or(missing_col1,missing_col2)] = imputed_data.cpu().numpy()[:,np.logical_or(missing_col1,missing_col2)]
        rawdata[np.logical_and(missing_row1,missing_row2)] = rawdata_pat3.copy()

    if j > 0:
      alist.append(rawdata.copy())
  return alist

## test

In [ ]:
Model_2_SD=[]
Model_2_SE_list=[]
Model_2_SE=0
Model_2_MSE=0
Model_2_estimate=0
Model_2_time=0

MI=3
batch_size=1024
ilr=1e-3 
clr=1e-3 
gamma=0.9
weight=0.1 
gp_lambda=10 
epochs=300

a0=-1
a1=-3
a2=3
a3=-1
a4=-3
a5=2
Monte_Carlo = 100


for iteration in range(1,1+Monte_Carlo):
  sample,missing_row1,missing_row2,missing_col1,missing_col2,y=data_G_linear(n=n,p=p,data=adni_data,seed=iteration,a0=a0,a1=a1,a2=a2,a3=a3,a4=a4,a5=a5)
  # print(sum(missing_row))
  truth1=sample[missing_row1==True,:][:,missing_col1==True].copy()
  truth2=sample[missing_row2==True,:][:,missing_col2==True].copy()
  
  mis1 = sample[:,missing_col1==True].copy()
  mis2 = sample[:,missing_col2==True].copy()
  mis1[missing_row1==True,:] = 0
  mis2[missing_row2==True,:] = 0
  sample[:,missing_col1==True]=mis1
  sample[:,missing_col2==True]=mis2
  rawdata = sample.copy()

  s=time.time()
  # alist=[]
  # for i in range(MI):
  #   data = mlp_MIGAN1_version2(rawdata.copy(),missing_row1,missing_row2,missing_col1,missing_col2,n=n,p_f=p,Batch_size=batch_size,ilr=ilr,clr=clr,gamma=gamma,weight=weight,gp_lambda=gp_lambda,epochs=epochs,MI_seed=MI)[0]
  #   alist.append(data)
  
  alist = mlp_MIGAN1_iterative2(rawdata.copy(),missing_row1,missing_row2,missing_col1,missing_col2,n=n,p_f=p,Batch_size=batch_size,ilr=ilr,clr=clr,gamma=gamma,weight=weight,gp_lambda=gp_lambda,epochs=epochs,MI=MI)
  
  mse,b,c,se=data_to_stats(alist,truth1,truth2,missing_row1,missing_row2,missing_col1,missing_col2)

  print('Model_2 cost ',time.time()-s,' sec \n','MSE:', mse,'; Estimate: ',b[1])
  Model_2_SE_list.append(se)
  Model_2_SE=(Model_2_SE*(iteration-1)+se)/iteration
  Model_2_SD.append(b[1])
  Model_2_MSE=(Model_2_MSE*(iteration-1)+mse)/iteration
  Model_2_estimate=(Model_2_estimate*(iteration-1)+b[1])/iteration
  Model_2_time = (Model_2_time*(iteration-1)+time.time()-s)/iteration

  if iteration%5==0:
    print('iteration: ',iteration, 'Model_2_MSE: ',Model_2_MSE,'Model_2_estimate: ',Model_2_estimate,'Model_2_SE: ',Model_2_SE)
    
    Model_2_StD=np.std(Model_2_SD, dtype=np.float64)
    print('Model_2_StD: ',Model_2_StD)
    # Model_2_SE_median=statistics.median(Model_2_SE_list)
    # print('Model_2_SE_median: ',Model_2_SE_median)
    print('Model_2 average cost:', Model_2_time/MI)

    print('n:', n, ' p: ',p, ' Monte Carlo:',Monte_Carlo, 'MI:',MI,'batch_size:',batch_size,'ilr:',ilr,'clr:',clr,'gamma:',gamma,'weight:',weight,'gp_lambda:',gp_lambda,'epochs:',epochs)
    print('a0=',a0,' a1=', a1,' a2=',a2,' a3=',a3,' a4=',a4,' a5=',a5, 'q=',q)